In [326]:
import numpy as np
import pandas as pd
import re
import datetime
import openpyxl
import calendar
import os
import glob
from pptx import Presentation
from pptx.util import Inches
from pptx.util import Pt
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor
from pptx.enum.text import MSO_AUTO_SIZE
from pptx.enum.shapes import MSO_SHAPE
import io
from functools import reduce
import math
import psutil

In [327]:
''' 
Список МО для выгрузки из отчета ЦБ Главное - Дополнительные отчеты - показатели записи на прием к врачу за период в 1 неделю которые у меня называются record
ГАУЗ ТО "Городская поликлиника № 1"; ГАУЗ ТО "Городская поликлиника № 12"; ГАУЗ ТО "Городская поликлиника № 13 "; ГАУЗ ТО "Городская поликлиника № 4"; ГАУЗ ТО "Городская поликлиника №5"; ГАУЗ ТО "Областная больница № 19" (г.Тюмень); ГАУЗ ТО «Городская Поликлиника № 6»; ГАУЗ ТО «Городская поликлиника № 8»; ГАУЗ ТО «Городская поликлиника №17»; ГАУЗ ТО «Городская поликлиника №3»; ГБУЗ ТО "Областная больница № 11" (р.п. Голышманово); ГБУЗ ТО "Областная больница № 12" (г.Заводоуковск); ГБУЗ ТО "Областная больница № 14 им. В.Н.Шанаурина" (с. Казанское); ГБУЗ ТО "Областная больница № 15" (с.Нижняя Тавда); ГБУЗ ТО "Областная больница № 23" (г.Ялуторовск); ГБУЗ ТО "Областная больница № 24" (с.Ярково); ГБУЗ ТО "Областная больница № 3" (г.Тобольск); ГБУЗ ТО "Областная больница № 4" (г.Ишим); ГБУЗ ТО "Областная больница № 9" (с.Вагай); ГБУЗ ТО "Областная больница №13" (с. Исетское); ГБУЗ ТО "Областная стоматологическая поликлиника"; ГБУЗ ТО "ОКБ №2"; ГБУЗ ТО "ОНД"; ГБУЗ ТО "Роддом № 3"; ГБУЗ ТО "Роддом №2"; ГБУЗ ТО «Областной клинический фтизиопульмонологический центр»; ГБУЗ ТО Областная больница №20 (с.Уват); ГБУЗ ТО ОКПБ
'''

' \nСписок МО для выгрузки из отчета ЦБ Главное - Дополнительные отчеты - показатели записи на прием к врачу за период в 1 неделю которые у меня называются record\nГАУЗ ТО "Городская поликлиника № 1"; ГАУЗ ТО "Городская поликлиника № 12"; ГАУЗ ТО "Городская поликлиника № 13 "; ГАУЗ ТО "Городская поликлиника № 4"; ГАУЗ ТО "Городская поликлиника №5"; ГАУЗ ТО "Областная больница № 19" (г.Тюмень); ГАУЗ ТО «Городская Поликлиника № 6»; ГАУЗ ТО «Городская поликлиника № 8»; ГАУЗ ТО «Городская поликлиника №17»; ГАУЗ ТО «Городская поликлиника №3»; ГБУЗ ТО "Областная больница № 11" (р.п. Голышманово); ГБУЗ ТО "Областная больница № 12" (г.Заводоуковск); ГБУЗ ТО "Областная больница № 14 им. В.Н.Шанаурина" (с. Казанское); ГБУЗ ТО "Областная больница № 15" (с.Нижняя Тавда); ГБУЗ ТО "Областная больница № 23" (г.Ялуторовск); ГБУЗ ТО "Областная больница № 24" (с.Ярково); ГБУЗ ТО "Областная больница № 3" (г.Тобольск); ГБУЗ ТО "Областная больница № 4" (г.Ишим); ГБУЗ ТО "Областная больница № 9" (с.Вагай); 

In [328]:
# Это дата выгрузки по МО из ФРМР
upload_date = '02.02.2024'

In [329]:
# Это дата выгрузки отчетов original и workers  по пятницам
period_last = '22.02.2024'

In [330]:
# Указываем верхнюю и нижнюю границу периода
start_date = '2024-02-19'
end_date = '2024-02-25'

In [331]:
period = f"{start_date} - {end_date}"

In [332]:
period

'2024-02-19 - 2024-02-25'

In [333]:
# Переменная окружения
ROOT_PATH = os.getcwd()
# поправочный файл от Анны Флягиной в котором учет должности ведется в перекрестной таблице
POST_AD = ROOT_PATH + '/guide/uchet_doljnosty.xlsx'
# в этой папке содержится отчет по штатным единицам из ФРМР по каждой МО они должны обновляться раз в неделю
FRMR_FILES = ROOT_PATH + '/frmr'
# в этой папке содержится справочник МО
GUIDE_MEDORGANIZATION = ROOT_PATH + '/guide/medicine_organization_2.xlsx'
# в этой папке содержится список исключений которые мы не принимаем в расчет файл формирует Анна Флягина
DELETE_ORGANIZATION = ROOT_PATH + '/guide/ОБЩИЙ ДЕКАБРЬ.xlsx'
# Это отчет о сотрудниках работающих в организации который формируется в ФРМР на дату аналогичную выгрузке из АНАЛИТ (например на 8 число для 1 слайда) для второго слайда - на 4 число, это другая выгрузка
WORKERS_XLS = ROOT_PATH + '/input/workers.xlsx'
#Это перечень должностей которые мы учитываем при подсчете
LIST_DOLJNOST = ROOT_PATH + '/guide/tablica_doljnost.xlsx'
# Это данные из Аналит-расписание на пятницу
ORIGINAL_ANALIT = ROOT_PATH + '/input/original.xlsx'
# Это данные из Аналит расписания на понедельник
FILTR_ANALIT = ROOT_PATH + '/input/original_filtr.xlsx'
# Это перечень строк из ФРМР которые не следует использовать при расчетах
GUIDE_DELETE_MED = ROOT_PATH + '/guide/guide_delete.xlsx'
# Это норма слотов для каждой должности
NORMA_SLOTOV = ROOT_PATH + '/guide/norma_slotov.xlsx'
# Это записи на прием к врачу
RECORD_DOCTORS = ROOT_PATH + '/input/record.xlsx'
# Это неотложка
EMERGENCY_XLSX = ROOT_PATH + '/emergency'
NEOTLOJKA_XLSX = ROOT_PATH + '/emergency/neotlojka.xlsx'
# Это три файла выгрузки для обработки аналит
ANALIT_SLIDE5 = ROOT_PATH + '/analitSlide5'
# Это расписание 1С
RASPISANIE_1C =  ROOT_PATH + '/input/1c_raspisanie.xlsx'
# Это справочник должностей и соответствующих им специальностей из аналит
DOLJNOST_ANALIT = ROOT_PATH + '/guide/analit_doljnost.xlsx'
# Это корректировочный файл для удаленя из 1С
DELETE_RAS_1C = ROOT_PATH + '/input/Del_ras1c.xlsx'
# Это еще один корректировочный файл для файло которые мы не берем из original_filtr
DELETE_ORIGINAL_FILTR = ROOT_PATH + '/guide/analit_original_filter_delete.xlsx'
# Это порядок расположения столбцов
SEQUENCE = pd.read_excel(ROOT_PATH + '/guide/psequence.xlsx')

##### Порядок отображения МО

In [334]:
sequence_mo = SEQUENCE

##### Настройка цветов презентации

In [335]:
color_red = RGBColor(246, 175, 171)
color_green = RGBColor(153, 219, 161)
color_neutral = RGBColor(223, 220, 230)
color_header = RGBColor(99, 89, 122)
color_title = RGBColor(55, 34, 105)
color_yallow = RGBColor(246, 235, 171)
# Задание черного цвета границы таблицы
border_color = RGBColor(0, 0, 0)  # Черный цвет (RGB: 0, 0, 0)

##### Подготавливаем данные для слайда 1, столбца 1

In [336]:
# Импортируем справочник медорганизаций
medorganization = pd.read_excel(GUIDE_MEDORGANIZATION)

In [337]:

# Загружаем выгрузки из ФРМР по МО и объединяем
my_excel = glob.glob(os.path.join(FRMR_FILES,"*.xlsx"))
len(my_excel)
svod = []
for my_files in my_excel:
    next_df = pd.read_excel(my_files, sheet_name=1)
    next_df['МО'] = my_files.split("/")[-1]
    svod.append(next_df)
frmr = pd.concat(svod, sort=False, axis=0)
frmr['МО'] = frmr['МО'].str.replace(r'.xlsx$', '', regex=True)
frmr = frmr[frmr['Должность'] != 'Итого:']
frmr = frmr.fillna(0)

In [338]:
frmr['Должность по Фед. справочнику'] = frmr['Должность по Фед. справочнику'].str.lower()

##### Фильтруем по должности

In [339]:
tablica_doljnost = pd.read_excel(LIST_DOLJNOST)
employee_list = tablica_doljnost['Должность'].to_list()
frmr_all = frmr[frmr['Должность'].isin(employee_list)] # Это фильтр по должности
frmr_all = frmr_all[frmr_all['Отделение стационара'] == 0] # Это исключение стационаров

In [340]:
# Это собранный Аней Флягиной список исключений
deletion = pd.read_excel(DELETE_ORGANIZATION, sheet_name=1, usecols=['Структурное подразделение', 'Должность', 'Должность по Фед. справочнику', 
                                                                     'Штатных единиц', 'Занятых штатных единиц', 'В т.ч. внешнее совместительство', 
                                                                     'Вакансий', 'MO']) 
deletion.rename(columns={'MO': 'МО'}, inplace=True)
deletion = deletion[['МО', 'Структурное подразделение', 'Должность', 'Должность по Фед. справочнику']]

In [341]:
# Мержим обработанную выгрузку из ФРМР с исключениями и убираем исключения
frmr_all = frmr_all.merge(deletion, on=['Структурное подразделение', 'Должность', 'Должность по Фед. справочнику', 'МО'], how='left', indicator=True)
frmr_all = frmr_all.query('_merge == "left_only"').drop('_merge', axis=1)
frmr_all.reset_index(drop=True, inplace=True)

In [342]:
test_frmr_all_ftisio_unique = frmr_all[frmr_all['МО'] == 'ОКФЦ']

##### Мержим обработанный список со справочником медорганизаций чтобы привести названия в нужный нам вид

In [343]:
frmr_all_merge = pd.merge(frmr_all, medorganization, on='МО', how='left')
frmr_all_merge = frmr_all_merge[['Организация', 'Структурное подразделение', 'Отделение стационара', 'Должность по Фед. справочнику', 'Штатных единиц', 'Занятых штатных единиц', 'В т.ч. внешнее совместительство', 'Вакансий']]
frmr_all_merge.rename(columns={'Должность по Фед. справочнику': 'Должность'}, inplace=True)

In [344]:
frmr_all_merge['Организация'].unique()

array([nan, 'ГАУЗ ТО «ГП №13»', 'ГБУЗ ТО «ОКФЦ»', 'ГАУЗ ТО "ОСП"',
       'ГБУЗ ТО «ОБ №23»', 'ГАУЗ ТО «ГП №17»', 'ГБУЗ ТО «ОКБ №2»',
       'ГБУЗ ТО «ОБ №13»', 'ГАУЗ ТО «ГП №4»', 'ГАУЗ ТО «ГП №1»',
       'ГБУЗ ТО «ОБ №12»', 'ГБУЗ ТО «Роддом № 2»', 'ГАУЗ ТО «ГП №6»',
       'ГАУЗ ТО «ГП №12»', 'ГБУЗ ТО «ОБ №15»', 'ГБУЗ ТО «ОБ №3»',
       'ГБУЗ ТО «ОНД»', 'ГБУЗ ТО «ОБ №19»', 'ГБУЗ ТО «ОБ №24»',
       'ГАУЗ ТО «ГП №3»', 'ГБУЗ ТО «Перинатальный центр»',
       'ГБУЗ ТО «ОБ №11»', 'ГБУЗ ТО «ОБ №4»', 'ГБУЗ ТО «ОБ №14»',
       'ГБУЗ ТО «ОБ №9»', 'ГАУЗ ТО «ГП №5»', 'ГАУЗ ТО «ГП №8»',
       'ГБУЗ ТО «Родильный дом №3»', 'ГБУЗ ТО «ОКПБ»', 'ГБУЗ ТО «ОБ №20»',
       'ГБУЗ ТО «ОКБ №1»'], dtype=object)

In [345]:
# Загрузка корректировочного файла от Анны Флягиной
uchet_doljnosty = pd.read_excel(POST_AD).fillna(0)
employee_uchet = pd.melt(uchet_doljnosty, id_vars=['Организация'], value_vars=employee_list, var_name='Должность', value_name='Проверка')
employee_uchet.sort_values(by='Проверка', ascending=True, inplace=True)

##### Объединяем с проверочным файлом в котором указаны должности по которым мы не считаем и удаляем ненужные

In [346]:
frmr_all_uchet = pd.merge(frmr_all_merge, employee_uchet, on=['Организация', 'Должность'], how='left').fillna(0)
frmr_all_uchet = frmr_all_uchet[frmr_all_uchet['Проверка'] != 0]

/tmp/ipykernel_7504/4235904968.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  frmr_all_uchet = pd.merge(frmr_all_merge, employee_uchet, on=['Организация', 'Должность'], how='left').fillna(0)


In [347]:
frmr_all_uchet['Организация'].unique()

array(['ГАУЗ ТО «ГП №13»', 'ГБУЗ ТО «ОКФЦ»', 'ГАУЗ ТО "ОСП"',
       'ГБУЗ ТО «ОБ №23»', 'ГАУЗ ТО «ГП №17»', 'ГБУЗ ТО «ОКБ №2»',
       'ГБУЗ ТО «ОБ №13»', 'ГАУЗ ТО «ГП №4»', 'ГАУЗ ТО «ГП №1»',
       'ГБУЗ ТО «ОБ №12»', 'ГБУЗ ТО «Роддом № 2»', 'ГАУЗ ТО «ГП №6»',
       'ГАУЗ ТО «ГП №12»', 'ГБУЗ ТО «ОБ №15»', 'ГБУЗ ТО «ОБ №3»',
       'ГБУЗ ТО «ОНД»', 'ГБУЗ ТО «ОБ №19»', 'ГБУЗ ТО «ОБ №24»',
       'ГАУЗ ТО «ГП №3»', 'ГБУЗ ТО «ОБ №11»', 'ГБУЗ ТО «ОБ №4»',
       'ГБУЗ ТО «ОБ №14»', 'ГБУЗ ТО «ОБ №9»', 'ГАУЗ ТО «ГП №5»',
       'ГАУЗ ТО «ГП №8»', 'ГБУЗ ТО «Родильный дом №3»', 'ГБУЗ ТО «ОКПБ»',
       'ГБУЗ ТО «ОБ №20»'], dtype=object)

In [348]:
proverka_all_uchet = frmr_all_uchet[frmr_all_uchet['Организация'] == 'ГБУЗ ТО «ОКФЦ»']

##### Расчет Слайд 1 Столбец 1

In [349]:
# формирование первого столбца первого слайда
frmr_all_slide_1 = frmr_all_uchet[['Организация', 'Должность']]

In [350]:
test_ftisiatr  = frmr_all_uchet[['Организация', 'Должность']]

In [351]:
test_ftisiatr['Организация'].unique()

array(['ГАУЗ ТО «ГП №13»', 'ГБУЗ ТО «ОКФЦ»', 'ГАУЗ ТО "ОСП"',
       'ГБУЗ ТО «ОБ №23»', 'ГАУЗ ТО «ГП №17»', 'ГБУЗ ТО «ОКБ №2»',
       'ГБУЗ ТО «ОБ №13»', 'ГАУЗ ТО «ГП №4»', 'ГАУЗ ТО «ГП №1»',
       'ГБУЗ ТО «ОБ №12»', 'ГБУЗ ТО «Роддом № 2»', 'ГАУЗ ТО «ГП №6»',
       'ГАУЗ ТО «ГП №12»', 'ГБУЗ ТО «ОБ №15»', 'ГБУЗ ТО «ОБ №3»',
       'ГБУЗ ТО «ОНД»', 'ГБУЗ ТО «ОБ №19»', 'ГБУЗ ТО «ОБ №24»',
       'ГАУЗ ТО «ГП №3»', 'ГБУЗ ТО «ОБ №11»', 'ГБУЗ ТО «ОБ №4»',
       'ГБУЗ ТО «ОБ №14»', 'ГБУЗ ТО «ОБ №9»', 'ГАУЗ ТО «ГП №5»',
       'ГАУЗ ТО «ГП №8»', 'ГБУЗ ТО «Родильный дом №3»', 'ГБУЗ ТО «ОКПБ»',
       'ГБУЗ ТО «ОБ №20»'], dtype=object)

In [352]:

frmr_all_slide_1.rename(columns={'Организация': 'Наименование МО'}, inplace=True)

/tmp/ipykernel_7504/1896369310.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frmr_all_slide_1.rename(columns={'Организация': 'Наименование МО'}, inplace=True)


In [353]:
frmr_all_slide_1['Должность'] = np.where(frmr_all_slide_1['Должность'] == 'врач-фтизиатр', 'врач-фтизиатр участковый', frmr_all_slide_1['Должность'])

/tmp/ipykernel_7504/1793884492.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frmr_all_slide_1['Должность'] = np.where(frmr_all_slide_1['Должность'] == 'врач-фтизиатр', 'врач-фтизиатр участковый', frmr_all_slide_1['Должность'])


In [354]:
frmr_all_slide_1['Должность'] = np.where(frmr_all_slide_1['Должность'] == 'врач-психиатр детский', 'врач-психиатр детский участковый', frmr_all_slide_1['Должность'])

/tmp/ipykernel_7504/2508088442.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frmr_all_slide_1['Должность'] = np.where(frmr_all_slide_1['Должность'] == 'врач-психиатр детский', 'врач-психиатр детский участковый', frmr_all_slide_1['Должность'])


In [355]:
double_frmr_slide_1 = frmr_all_slide_1.drop_duplicates()

In [356]:
slide_1_column1 = pd.pivot_table(double_frmr_slide_1, 
                                 index='Наименование МО', 
                                 values='Должность', 
                                 aggfunc='count', 
                                 margins=True, 
                                 margins_name='Итого').reset_index()

In [357]:
slide_1_column1.rename(columns={'Должность': 'Кол-во должностей из 14'}, inplace=True)
slide_1_column1 = slide_1_column1[(slide_1_column1['Наименование МО'] != 'ГБУЗ ТО «ОКБ №1»') &   
                                  (slide_1_column1['Наименование МО'] != 'ГБУЗ ТО «Перинатальный центр»')]
slide_1_column1.reset_index(drop=True, inplace=True)

##### СЛАЙД 1 СТОЛБЕЦ 2

In [358]:
workers = pd.read_excel(WORKERS_XLS, skiprows=6)
workers.rename(columns={'Unnamed: 0': 'СНИЛС', 
                    'Unnamed: 1': 'Фамилия', 
                    'Unnamed: 2': 'Имя', 
                    'Unnamed: 3': 'Отчество', 
                    'Unnamed: 4': 'Пол', 
                    'Unnamed: 5': 'Дата рождения', 
                    'Unnamed: 6': 'Сведения о трудоустройстве'}, inplace=True)

/home/serg/prezentation/venv/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [359]:
workers['Должность по фед справочнику'] = workers['Должность по фед справочнику'].str.lower()

In [360]:
post_array = workers['Должность по фед справочнику'].unique()
post_df = pd.DataFrame({'Должность':post_array})

In [361]:
guide_post = pd.merge(tablica_doljnost, post_df, on='Должность', how='inner')
guide_post = guide_post.rename(columns={'Должность' : 'Должность по фед справочнику'})

In [362]:
# Оставляем только амбулаторные подразделения
workers = workers[workers['Тип структурного подразделения'] == 'Амбулаторный']
# Подгружаем справочник на удаление
delete_rows = pd.read_excel(GUIDE_DELETE_MED, skiprows=6)
delete_rows.rename(columns={'Unnamed: 0': 'СНИЛС', 
                    'Unnamed: 1': 'Фамилия', 
                    'Unnamed: 2': 'Имя', 
                    'Unnamed: 3': 'Отчество', 
                    'Unnamed: 4': 'Пол', 
                    'Unnamed: 5': 'Дата рождения', 
                    'Unnamed: 6': 'Сведения о трудоустройстве'}, inplace=True)
delete_rows = delete_rows[['Краткое наименование организации', 'OID структурного подразделения', 'Должность по фед справочнику']]
delete_rows['Удалить'] = 'True'
# Мержим со справочником на удаление и удаляем все значения не соответсвующие типу амбулаторный из исходного массива
result = pd.merge(workers, delete_rows,on=['Краткое наименование организации', 
                                      'OID структурного подразделения', 
                                      'Должность по фед справочнику'], how='left')
result = result[result['Удалить'].isnull()]


In [363]:
# Удаление дубликатов из первого DataFrame
guide_post['Delete'] = True
result_filtered = pd.merge(result, guide_post, on=['Должность по фед справочнику'], how='left')


In [364]:
result_filtered = result_filtered[result_filtered['Delete']  == True]

In [365]:
resultSlide = pd.merge(result_filtered, medorganization, on='Краткое наименование организации', how='left')

In [366]:
resultSlide.drop('Должность', axis=1, inplace=True)

In [367]:
resultSlide.rename(columns={'Должность по фед справочнику': 'Должность'}, inplace=True)

In [368]:
itog1 = pd.pivot_table(resultSlide, 
                      index=['Организация', 'Должность'], 
                      columns=['Причина временного неисполнения функциональных обязанностей'], 
                      values='СНИЛС', 
                      aggfunc=pd.Series.nunique).fillna(0)
itog1.reset_index(inplace=True)

In [369]:
itog1_slide1 = pd.merge(itog1, employee_uchet, on=['Организация', 'Должность'], how='left').fillna(0)

In [370]:
itog1_slide1 = itog1_slide1[itog1_slide1['Проверка'] != 0]

In [371]:
itog2 = pd.pivot_table(resultSlide, 
                      index=['Организация', 'Должность'], 
                      values='СНИЛС', 
                      aggfunc=pd.Series.nunique)
itog2.reset_index(inplace=True)

In [372]:
itog2_slide1 = pd.merge(itog2, employee_uchet, on=['Организация', 'Должность'], how='left').fillna(0)

In [373]:
itog2_slide1[itog2_slide1['Организация'] == 'ГБУЗ ТО «ОНД»']

,Организация,Должность,СНИЛС,Проверка
245,ГБУЗ ТО «ОНД»,врач-психиатр-нарколог,13,1.0


In [374]:
itog = pd.merge(itog2_slide1, itog1_slide1, on=['Организация', 'Должность', 'Проверка'], how='outer').fillna(0)

In [375]:
guide_post2 = pd.merge(tablica_doljnost, post_df, on='Должность', how='inner')

In [376]:
itog_merge = pd.merge(guide_post2, itog, on=['Должность'], how='left')

In [377]:
itog_merge = itog_merge[itog_merge['Проверка'] != 0]

In [378]:
itog_merge_mo = pd.merge(medorganization, itog_merge, on='Организация', how='left').dropna()

In [379]:
itog_merge_mo = itog_merge_mo[['Организация', 'Должность', 'СНИЛС', 'временная нетрудоспособность', 
                               'дополнительное профессиональное образование', 'иные причины', 'отпуск без сохранения заработной платы', 
                               'отпуск по беременности и родам', 'отпуск по уходу за ребенком']]

In [380]:
itog_merge_mo[itog_merge_mo['Организация'] == 'ГБУЗ ТО «ОКПБ»']

,Организация,Должность,СНИЛС,временная нетрудоспособность,дополнительное профессиональное образование,иные причины,отпуск без сохранения заработной платы,отпуск по беременности и родам,отпуск по уходу за ребенком
220,ГБУЗ ТО «ОКПБ»,врач-психиатр детский участковый,7.0,0.0,0.0,0.0,0.0,0.0,1.0


In [381]:
res = itog_merge_mo.sort_values(by=['Организация', 'Должность']).reset_index(drop=True)

In [382]:
res.rename(columns={'СНИЛС' : 'Всего'}, inplace=True)

In [383]:
res['Итого'] = res['Всего'] - res['временная нетрудоспособность'] - res['дополнительное профессиональное образование'] - res['иные причины'] - res['отпуск без сохранения заработной платы'] - res['отпуск по беременности и родам'] - res['отпуск по уходу за ребенком']
slide_1_column2 = pd.pivot_table(res, 
                                 index='Организация', 
                                 values='Всего', 
                                 aggfunc=np.sum, 
                                 margins=True, 
                                 margins_name='Итого').reset_index()

/tmp/ipykernel_7504/1761294361.py:2: FutureWarning: The provided callable <function sum at 0x7eff90181d80> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  slide_1_column2 = pd.pivot_table(res,


In [384]:
slide_1_column2 = pd.merge(medorganization, slide_1_column2, on='Организация', how='outer')

In [385]:
slide_1_column2 = slide_1_column2[(slide_1_column2['Краткое наименование организации'] != 'Тюменская больница ФГБУЗ ЗСМЦ ФМБА России') & 
                                  (slide_1_column2['Краткое наименование организации'] != 'Тобольская больница ФГБУЗ ЗСМЦ ФМБА России') & 
                                  (slide_1_column2['Краткое наименование организации'] != 'ГБУЗ ТО "ЦПБС"') & 
                                  (slide_1_column2['Краткое наименование организации'] != 'ГАУЗ ТО «Областной лечебно-реабилитационный центр»') & 
                                  (slide_1_column2['Краткое наименование организации'] != 'ГАУЗ ТО «Госпиталь для ветеранов войн»') & 
                                  (slide_1_column2['Краткое наименование организации'] != 'ГАУЗ ТО "ОБЛАСТНОЙ ОФТАЛЬМОЛОГИЧЕСКИЙ ДИСПАНСЕР"') & 
                                  (slide_1_column2['Краткое наименование организации'] != 'ГАУЗ ТО "МКМЦ "МЕДИЦИНСКИЙ ГОРОД"') & 
                                  (slide_1_column2['Краткое наименование организации'] != 'ГАУЗ ТО "МКДЦ"') & 
                                  (slide_1_column2['Краткое наименование организации'] != 'ГАУЗ ТО "ДЛРЦ "НАДЕЖДА"') &
                                  (slide_1_column2['Краткое наименование организации'] != 'ГБУЗ ТО «ОКБ № 1»') &
                                  (slide_1_column2['Краткое наименование организации'] != 'АО "МСЧ "НЕФТЯНИК"') &
                                  (slide_1_column2['Краткое наименование организации'] != 'ГБУЗ ТО "ПЕРИНАТАЛЬНЫЙ ЦЕНТР" (Г.ТЮМЕНЬ)')]

In [386]:
slide_1_column2 = slide_1_column2[['Организация', 'Всего']]
slide_1_column2.reset_index(drop=True, inplace=True)
slide_1_column2.rename(columns={'Организация': 'Наименование МО'}, inplace=True)
slide_1_column2['Наименование МО'] = slide_1_column2['Наименование МО'].fillna('Итого')

##### Слайд 1 Столбец 3

In [387]:
slide_1_column3 = pd.pivot_table(res, 
                                 index='Организация', 
                                 values='Итого', 
                                 aggfunc=np.sum, 
                                 margins=True, 
                                 margins_name='Итого').reset_index()
slide_1_column3 = pd.merge(medorganization, slide_1_column3, on='Организация', how='outer')
slide_1_column3 = slide_1_column3[(slide_1_column3['Краткое наименование организации'] != 'Тюменская больница ФГБУЗ ЗСМЦ ФМБА России') & 
                                  (slide_1_column3['Краткое наименование организации'] != 'Тобольская больница ФГБУЗ ЗСМЦ ФМБА России') & 
                                  (slide_1_column3['Краткое наименование организации'] != 'ГБУЗ ТО "ЦПБС"') & 
                                  (slide_1_column3['Краткое наименование организации'] != 'ГАУЗ ТО «Областной лечебно-реабилитационный центр»') & 
                                  (slide_1_column3['Краткое наименование организации'] != 'ГАУЗ ТО «Госпиталь для ветеранов войн»') & 
                                  (slide_1_column3['Краткое наименование организации'] != 'ГАУЗ ТО "ОБЛАСТНОЙ ОФТАЛЬМОЛОГИЧЕСКИЙ ДИСПАНСЕР"') & 
                                  (slide_1_column3['Краткое наименование организации'] != 'ГАУЗ ТО "МКМЦ "МЕДИЦИНСКИЙ ГОРОД"') & 
                                  (slide_1_column3['Краткое наименование организации'] != 'ГАУЗ ТО "МКДЦ"') & 
                                  (slide_1_column3['Краткое наименование организации'] != 'ГАУЗ ТО "ДЛРЦ "НАДЕЖДА"') &
                                  (slide_1_column3['Краткое наименование организации'] != 'ГБУЗ ТО «ОКБ № 1»') &
                                  (slide_1_column3['Краткое наименование организации'] != 'АО "МСЧ "НЕФТЯНИК"') &
                                  (slide_1_column3['Краткое наименование организации'] != 'ГБУЗ ТО "ПЕРИНАТАЛЬНЫЙ ЦЕНТР" (Г.ТЮМЕНЬ)')]
slide_1_column3 = slide_1_column3[['Организация', 'Итого']]
slide_1_column3.reset_index(drop=True, inplace=True)
slide_1_column3.rename(columns={'Организация': 'Наименование МО'}, inplace=True)
slide_1_column3['Наименование МО'] = slide_1_column3['Наименование МО'].fillna('Итого')

/tmp/ipykernel_7504/3511262596.py:1: FutureWarning: The provided callable <function sum at 0x7eff90181d80> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  slide_1_column3 = pd.pivot_table(res,


###### Создаем слайд 1 столбец 4
* Файл original выгружается из аналит расписания за период в одну неделю вперед по пути Статистика Здравоохранение - Организация работы фронт-офиса - Деталзация расписания врачей.

In [388]:
original = pd.read_excel(ORIGINAL_ANALIT)

In [389]:
original = original[(original['Медицинская организация'] != 'Всего')]
analit_merge = pd.merge(original, medorganization, on='Медицинская организация', how='left')
analit_result  = analit_merge[['Медицинская организация', 'Краткое наименование организации', 'Отделение', 'Отделение (адрес)',
       'Должность', 'Специальность', 'Врач', 'Дней расписания', 'Слотов на 7 дней', 'Слотов на 14 дней', 'Слотов на 30 дней',
       'Формиров. расписания на 7 дней, %', 'Формиров. расписания на 14 дней, %', 'Формиров. расписания на 30 дней, %',
       'Слотов первичного приёма на 7 дней', 'Доля первичного приёма на 7 дней, %', 'Слотов первичного приёма на 14 дней',
       'Доля первичного приёма на 14 дней, %', 'Слотов первичного приёма на 30 дней', 'Доля первичного приёма на 30 дней, %']]
delete_rows2 = pd.read_excel(GUIDE_DELETE_MED, skiprows=6)
delete_rows2.rename(columns={'Unnamed: 0': 'СНИЛС', 
                    'Unnamed: 1': 'Фамилия', 
                    'Unnamed: 2': 'Имя', 
                    'Unnamed: 3': 'Отчество', 
                    'Unnamed: 4': 'Пол', 
                    'Unnamed: 5': 'Дата рождения', 
                    'Unnamed: 6': 'Сведения о трудоустройстве'}, inplace=True)
delete_rows2 = delete_rows2[['Краткое наименование организации', 'Наименование структурного подразделения', 'Должность по фед справочнику']]
delete_rows2 = delete_rows2.rename(columns={'Должность по фед справочнику' : 'Должность', 'Наименование структурного подразделения': 'Отделение'})
delete_rows2['Удалить'] = 'True'
result_original = pd.merge(analit_result, delete_rows2,on=['Краткое наименование организации', 
                                      'Отделение', 
                                      'Должность'], how='left')
result_original = result_original[result_original['Удалить'] != 'True']

###### Порядок работы с данными
* Вначале смержить со справочником медорганизаций
* потом смержить с таблицей исключений
* Потом убрать исключения
* Только затем мержить с остальными данными

In [390]:
resultSlide1_4 = pd.merge(result_original, medorganization, on=['Медицинская организация', 'Краткое наименование организации'], how='outer')[[ 'Организация', 'Отделение', 'Отделение (адрес)', 'Должность', 'Специальность', 'Врач',
'Дней расписания', 'Слотов на 7 дней', 'Слотов на 14 дней',
'Слотов на 30 дней', 'Формиров. расписания на 7 дней, %',
'Формиров. расписания на 14 дней, %',
'Формиров. расписания на 30 дней, %',
'Слотов первичного приёма на 7 дней',
'Доля первичного приёма на 7 дней, %',
'Слотов первичного приёма на 14 дней',
'Доля первичного приёма на 14 дней, %',
'Слотов первичного приёма на 30 дней',
'Доля первичного приёма на 30 дней, %', 'Удалить']]

In [391]:
resultSlide1_4 = pd.merge(resultSlide1_4, employee_uchet, on=['Организация', 'Должность'], how='left').fillna(0)

/tmp/ipykernel_7504/4259935434.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  resultSlide1_4 = pd.merge(resultSlide1_4, employee_uchet, on=['Организация', 'Должность'], how='left').fillna(0)


In [392]:
resultSlide1_4 = resultSlide1_4[resultSlide1_4['Проверка'] != 0]

In [393]:
table_analit_doljnost = pd.read_excel(DOLJNOST_ANALIT)
merge = pd.merge(resultSlide1_4, table_analit_doljnost, on = ['Должность', 'Специальность'], how='right')

In [394]:
df1 = merge
df2 = merge
df1['Доля первичного приёма на 7 дней, %'] = df1['Доля первичного приёма на 7 дней, %'].replace('-', 0)
df1['Доля первичного приёма на 14 дней, %'] = df1['Доля первичного приёма на 14 дней, %'].replace('-', 0)
df1['Доля первичного приёма на 30 дней, %'] = df1['Доля первичного приёма на 30 дней, %'].replace('-', 0)
svod_slotov = pd.pivot_table(df1, values=['Слотов на 7 дней', 
                                          'Слотов на 14 дней', 
                                          'Слотов на 30 дней', 
                                          'Слотов первичного приёма на 7 дней', 
                                          'Слотов первичного приёма на 14 дней', 
                                          'Слотов первичного приёма на 30 дней'], 
                             index=['Организация', 'Должность'], 
                             aggfunc=np.sum, 
                             fill_value=0)
svod_slotov.reset_index(drop=False, inplace=True)
def proz7days(row):
    if row['Слотов первичного приёма на 7 дней'] != 0 and row['Слотов на 7 дней'] != 0:
        val = round(row['Слотов первичного приёма на 7 дней'] / row['Слотов на 7 дней'] * 100)
    else:
        val = 0
    return val
svod_slotov['% конкурентных слотов на 7 дней'] = svod_slotov.apply(proz7days, axis = 1)
def proz14days(row):
    if row['Слотов первичного приёма на 14 дней'] != 0 and row['Слотов на 14 дней'] != 0:
        val = round(row['Слотов первичного приёма на 14 дней'] / row['Слотов на 14 дней'] * 100)
    else:
        val = 0
    return val
svod_slotov['% конкурентных слотов на 14 дней'] = svod_slotov.apply(proz14days, axis = 1)
def proz30days(row):
    if row['Слотов первичного приёма на 30 дней'] != 0 and row['Слотов на 30 дней'] != 0:
        val = round(row['Слотов первичного приёма на 30 дней'] / row['Слотов на 30 дней'] * 100)
    else:
        val = 0
    return val
svod_slotov['% конкурентных слотов на 30 дней'] = svod_slotov.apply(proz30days, axis = 1)
df2_7days = df2[df2['Доля первичного приёма на 7 дней, %'] != '-']
df2_14days = df2[df2['Доля первичного приёма на 14 дней, %'] != '-']
df2_30days = df2[df2['Доля первичного приёма на 30 дней, %'] != '-']
unigue_7days = pd.pivot_table(df2_7days, values=['Врач'], 
                             index=['Организация', 'Должность'], 
                             aggfunc=pd.Series.nunique, 
                             fill_value=0)
unigue_14days = pd.pivot_table(df2_14days, values=['Врач'], 
                             index=['Организация', 'Должность'], 
                             aggfunc=pd.Series.nunique, 
                             fill_value=0)
unigue_30days = pd.pivot_table(df2_30days, values=['Врач'], 
                             index=['Организация', 'Должность'], 
                             aggfunc=pd.Series.nunique, 
                             fill_value=0)
unigue_7days.reset_index(drop=False, inplace=True)
unigue_14days.reset_index(drop=False, inplace=True)
unigue_30days.reset_index(drop=False, inplace=True)
unigue_7days.rename(columns = {'Врач': 'кол-во сотрудников с сформированным расписанием на 7 дней'}, inplace=True)
unigue_14days.rename(columns = {'Врач': 'кол-во сотрудников с сформированным расписанием на 14 дней'}, inplace=True)
unigue_30days.rename(columns = {'Врач': 'кол-во сотрудников с сформированным расписанием на 30 дней'}, inplace=True)
all_days = [svod_slotov, unigue_7days, unigue_14days, unigue_30days]
itog = reduce(lambda left,right: pd.merge(left,right,on=['Организация', 'Должность'], how='outer'), all_days).fillna('none')

/tmp/ipykernel_7504/515607811.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df1['Доля первичного приёма на 7 дней, %'] = df1['Доля первичного приёма на 7 дней, %'].replace('-', 0)
/tmp/ipykernel_7504/515607811.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df1['Доля первичного приёма на 14 дней, %'] = df1['Доля первичного приёма на 14 дней, %'].replace('-', 0)
/tmp/ipykernel_7504/515607811.py:6: FutureWarning: The provided callable <function sum at 0x7eff90181d80> is currently using DataFrameGroupBy.sum. In a future ver

In [395]:
employee_with_schedule =itog.copy()

In [396]:
employee_with_schedule = employee_with_schedule[['Организация', 'Должность', 'кол-во сотрудников с сформированным расписанием на 7 дней', 'Слотов на 7 дней', 'Слотов первичного приёма на 7 дней']]

In [397]:
itog = itog[['Организация', 'кол-во сотрудников с сформированным расписанием на 7 дней']]
itog.rename(columns={'кол-во сотрудников с сформированным расписанием на 7 дней': 'Кол-во физ-лиц на которые заведено расписание'}, inplace=True)

In [398]:
slide1_column4 = pd.pivot_table(itog, index='Организация', values=['Кол-во физ-лиц на которые заведено расписание'], aggfunc=np.sum).reset_index()
slide1_column4.rename(columns={'Организация': 'Наименование МО'}, inplace=True)

/tmp/ipykernel_7504/3990487209.py:1: FutureWarning: The provided callable <function sum at 0x7eff90181d80> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  slide1_column4 = pd.pivot_table(itog, index='Организация', values=['Кол-во физ-лиц на которые заведено расписание'], aggfunc=np.sum).reset_index()


In [399]:
# Добавление строки "Итого" с числовым индексом
total_row = pd.Series(['Итого', slide1_column4['Кол-во физ-лиц на которые заведено расписание'].sum()], index=slide1_column4.columns)
slide1_column4 = pd.concat([slide1_column4, pd.DataFrame([total_row])], ignore_index=True)
# slide1_column4 = slide1_column4.append(total_row, ignore_index=True)

##### Создаем Слайд 1 столбец 6 Здесь мы считаем тех кто в отпуске по беременности и родам

In [400]:
decree = res[['Организация', 'отпуск по беременности и родам', 'отпуск по уходу за ребенком']]

In [401]:
decree['Кол-во физ-лиц в ФРМР в ДО'] = decree['отпуск по беременности и родам'] + decree['отпуск по уходу за ребенком']

/tmp/ipykernel_7504/1532569484.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  decree['Кол-во физ-лиц в ФРМР в ДО'] = decree['отпуск по беременности и родам'] + decree['отпуск по уходу за ребенком']


In [402]:
slide_1_column6 = pd.pivot_table(decree, 
                                 index='Организация', 
                                 values='Кол-во физ-лиц в ФРМР в ДО', 
                                 aggfunc=np.sum, 
                                 margins=True, 
                                 margins_name='Итого').reset_index()
slide_1_column6.rename(columns={'Организация': 'Наименование МО'}, inplace=True)

/tmp/ipykernel_7504/3786653454.py:1: FutureWarning: The provided callable <function sum at 0x7eff90181d80> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  slide_1_column6 = pd.pivot_table(decree,


##### Считаем 7 столбец 1 слайда

In [403]:
slide_1_column7 = pd.pivot_table(frmr_all_uchet, index='Организация', values='Занятых штатных единиц', aggfunc=np.sum, margins=True, margins_name='Итого').round(2).reset_index()

/tmp/ipykernel_7504/2237574477.py:1: FutureWarning: The provided callable <function sum at 0x7eff90181d80> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  slide_1_column7 = pd.pivot_table(frmr_all_uchet, index='Организация', values='Занятых штатных единиц', aggfunc=np.sum, margins=True, margins_name='Итого').round(2).reset_index()


In [404]:
slide_1_column7.rename(columns={'Организация': 'Наименование МО', 'Занятых штатных единиц': 'Кол-во ставок в ШР МО'}, inplace=True)

In [405]:
slide_1_column7['Кол-во ставок в ШР МО'] = slide_1_column7['Кол-во ставок в ШР МО'].round(2)

#### Подготовка данных для слайда 2

1. Рассчитать долю фактического количества слотов от расчетных %
2. Рассчитать долю фактического количества слотов от расчетных у врачей участковой службы, %
3. Рассчитать долю конкурентных слотов от  общего количества слотов, %
4. Рассчитать доля конкурентных слотов участковой службы от общего количества слотов, %
5. Рассчитать итоги для каждого столбца с долей
6. Объединить в один датафрейм
7. Датафрейм использовать для построения таблицы и добавить таблицу на слайд
8. Настроить форматирование по условиям в таблице
9. Настроить оформление слайда

In [406]:
frmr_all_slide_2 = frmr_all_uchet
frmr_all_slide_2 = frmr_all_slide_2[['Организация', 'Структурное подразделение', 'Отделение стационара', 'Должность', 'Штатных единиц', 'Занятых штатных единиц', 'В т.ч. внешнее совместительство']]
norma = pd.read_excel(NORMA_SLOTOV)
frmr_all_slide_2 = pd.merge(frmr_all_uchet, norma, how='left', on='Должность')
frmr_all_slide_2 = frmr_all_slide_2[frmr_all_slide_2['Отделение стационара'] == 0]
frmr_all_slide_2['Сумма слотов на врача'] = frmr_all_slide_2['Занятых штатных единиц'] * frmr_all_slide_2['Норма слотов']
pivot_table_frmr = pd.pivot_table(frmr_all_slide_2, index='Организация', values='Сумма слотов на врача', aggfunc=np.sum, margins=True, margins_name='Итого')
pivot_table_frmr = pivot_table_frmr.reset_index()

/tmp/ipykernel_7504/2565044582.py:7: FutureWarning: The provided callable <function sum at 0x7eff90181d80> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot_table_frmr = pd.pivot_table(frmr_all_slide_2, index='Организация', values='Сумма слотов на врача', aggfunc=np.sum, margins=True, margins_name='Итого')


#### Обрабатываю данные показателей записи на прием к врачу

In [407]:
indi = pd.read_excel(RECORD_DOCTORS, skiprows=[0, 1, 2, 3, 4])

In [408]:
indicators = indi.copy()

In [409]:
indicators.rename(columns={'Unnamed: 0': 'МО', 'Unnamed: 3': 'Должность'}, inplace=True)
indicators = indicators[['МО', 'Должность', 'Общее число слотов в расписании']].fillna(0)
indicators.rename(columns={'МО': 'Медицинская организация'}, inplace=True)

In [410]:
indicator_mo = pd.merge(indicators, medorganization, how='left', on='Медицинская организация').dropna()

In [411]:
indicator_mo = indicator_mo[['Организация', 'Должность', 'Общее число слотов в расписании']]

In [412]:
indicator_mo = indicator_mo[(indicator_mo['Должность'] == 'врач - детский хирург') | 
            (indicator_mo['Должность'] == 'врач общей практики (семейный врач)') |
            (indicator_mo['Должность'] ==  'врач-акушер-гинеколог') |
            (indicator_mo['Должность'] ==  'врач-оториноларинголог') |
            (indicator_mo['Должность'] ==  'врач-офтальмолог') |
            (indicator_mo['Должность'] ==  'врач-педиатр участковый') |
            (indicator_mo['Должность'] ==  'врач-психиатр детский') |
            (indicator_mo['Должность'] ==  'врач-психиатр-нарколог') |
            (indicator_mo['Должность'] ==  'врач-стоматолог') |
            (indicator_mo['Должность'] ==  'врач-стоматолог детский') |
            (indicator_mo['Должность'] ==  'врач-стоматолог-терапевт') |
            (indicator_mo['Должность'] ==  'врач-терапевт участковый') |
            (indicator_mo['Должность'] ==  'врач-фтизиатр') |
            (indicators['Должность'] ==  'врач-фтизиатр участковый') |
            (indicator_mo['Должность'] ==  'врач-хирург') |
            (indicator_mo['Должность'] ==  'врач-психиатр детский участковый')]

/tmp/ipykernel_7504/193165115.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  indicator_mo = indicator_mo[(indicator_mo['Должность'] == 'врач - детский хирург') |


##### Проверка с таблицей с единичками по должностям, исключение из расчета должности которые считать не следует

In [413]:
indicator_mo_uchet = pd.merge(indicator_mo, employee_uchet, on=['Организация', 'Должность'], how='left').fillna(0)

In [414]:
indicator_mo_uchet = indicator_mo_uchet[indicator_mo_uchet['Проверка'] != 0]

1. Расчет показателя 'Доля фактического количества слотов от расчетных, %'

In [415]:
fact_slots = pd.pivot_table(indicator_mo_uchet, index='Организация', values='Общее число слотов в расписании', aggfunc=np.sum, margins=True, margins_name='Итого').reset_index()

/tmp/ipykernel_7504/2200017004.py:1: FutureWarning: The provided callable <function sum at 0x7eff90181d80> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  fact_slots = pd.pivot_table(indicator_mo_uchet, index='Организация', values='Общее число слотов в расписании', aggfunc=np.sum, margins=True, margins_name='Итого').reset_index()


In [416]:
itog_slots = pd.merge(fact_slots, pivot_table_frmr, on='Организация')

In [417]:
itog_slots['Доля фактического количества слотов от расчетных, не менее 80%'] = round(itog_slots['Общее число слотов в расписании'] / itog_slots['Сумма слотов на врача'] * 100, 2)
itog_slots = itog_slots[['Организация', 'Доля фактического количества слотов от расчетных, не менее 80%']]
itog_slots = itog_slots[itog_slots['Организация'] != 'ПЦ_Тюмень']
itog_slots = itog_slots[itog_slots['Организация'] != 'ОКБ1']
itog_slots.reset_index(drop=True, inplace=True)

2. Расчет показателя 'Доля фактического количества слотов от расчетных у врачей участковой службы, %'

##### считаем норму для участковой службы

In [418]:
frmr_uchastkov = frmr_all_merge[(frmr_all_merge['Должность'] ==  'врач-педиатр участковый') |  (frmr_all_merge['Должность'] ==  'врач-терапевт участковый')]

In [419]:
frmr_uchastkov = pd.merge(frmr_uchastkov, norma, how='left', on='Должность')

In [420]:
frmr_uchastkov['Норма штатных единиц'] = frmr_uchastkov['Норма слотов'] * frmr_uchastkov['Занятых штатных единиц']

In [421]:
pivot_table_frmr_uchastkov = pd.pivot_table(frmr_uchastkov, index='Организация', values='Норма штатных единиц', aggfunc=np.sum, margins=True, margins_name='Итого')
pivot_table_frmr_uchastkov = pivot_table_frmr_uchastkov.reset_index()

/tmp/ipykernel_7504/4131419777.py:1: FutureWarning: The provided callable <function sum at 0x7eff90181d80> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot_table_frmr_uchastkov = pd.pivot_table(frmr_uchastkov, index='Организация', values='Норма штатных единиц', aggfunc=np.sum, margins=True, margins_name='Итого')


In [422]:
pivot_table_frmr_uchastkov = pivot_table_frmr_uchastkov[['Организация', 'Норма штатных единиц']]

###### считаем факт для участковой службы

In [423]:
indicator_uchaskov = indicator_mo[(indicator_mo['Должность'] ==  'врач-педиатр участковый') |  (indicator_mo['Должность'] ==  'врач-терапевт участковый')]

In [424]:
indicator_mo_uchet_uchastkov = pd.merge(indicator_uchaskov, employee_uchet, on=['Организация', 'Должность'], how='left').fillna(0)

In [425]:
indicator_mo_uchet_uchastkov = indicator_mo_uchet_uchastkov[indicator_mo_uchet_uchastkov['Проверка'] != 0]

In [426]:
fact_slots_uchaskov = pd.pivot_table(indicator_mo_uchet_uchastkov, index='Организация', values='Общее число слотов в расписании', aggfunc=np.sum, margins=True, margins_name='Итого').reset_index()

/tmp/ipykernel_7504/3153925833.py:1: FutureWarning: The provided callable <function sum at 0x7eff90181d80> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  fact_slots_uchaskov = pd.pivot_table(indicator_mo_uchet_uchastkov, index='Организация', values='Общее число слотов в расписании', aggfunc=np.sum, margins=True, margins_name='Итого').reset_index()


In [427]:
itog_slots_uchastkov = pd.merge(fact_slots_uchaskov, pivot_table_frmr_uchastkov, on='Организация')
itog_slots_uchastkov['Доля фактического количества слотов от расчетных у врачей участковой службы, не менее 80%'] = round(itog_slots_uchastkov['Общее число слотов в расписании'] / itog_slots_uchastkov['Норма штатных единиц'] * 100, 2)
itog_slots_uchastkov = itog_slots_uchastkov[['Организация', 'Доля фактического количества слотов от расчетных у врачей участковой службы, не менее 80%']]
itog_slots_uchastkov.tail(1)

,Организация,"Доля фактического количества слотов от расчетных у врачей участковой службы, не менее 80%"
22,Итого,64.99


###### Слайд  2.2.Считаем долю конкурентных слотов от общего количества слотов, %

In [428]:
# Загружаю расписание 1С
# guide_mo_1c = pd.read_excel(GUIDE_MEDORGANIZATION)
ras_1c = pd.read_excel(RASPISANIE_1C, usecols=['region', 'mo_oid', 'mo_short_name', 'sp_oid', 'sp_name', 'mp_id', 
                                              'mp_fio', 'mp_dolgnost', 'mp_stavka', 'slots_type', 'date_report',
                                              'slots_referral', 'slot_lenght', 'slots_created', 'slots_booked',
                                              'slots_free_after_booked', 'visits_absence', 'visits_success'])

In [429]:
# Загружаю поправочную таблицу 
delRas1c = pd.read_excel(DELETE_RAS_1C, usecols=['region', 'mo_oid', 'mo_short_name', 'sp_oid', 'sp_name', 'mp_id', 
                                              'mp_fio', 'mp_dolgnost', 'mp_stavka', 'slots_type',
                                              'slots_referral', 'slot_lenght', 'slots_created', 'slots_booked',
                                              'slots_free_after_booked', 'visits_absence', 'visits_success'])

In [430]:
# Объединяю таблицы
ras1c_merged = pd.merge(ras_1c, delRas1c, on=['region', 'mo_oid', 'mo_short_name', 'sp_oid', 'sp_name', 'mp_id', 
                                              'mp_fio', 'mp_dolgnost', 'mp_stavka', 'slots_type',
                                              'slots_referral', 'slot_lenght', 'slots_created', 'slots_booked',
                                              'slots_free_after_booked', 'visits_absence', 'visits_success'], how='left', indicator=True)

In [431]:
ras1c_filtered = ras1c_merged[ras1c_merged['_merge'] == 'left_only'].drop(columns='_merge')

In [432]:
primer  = ras1c_filtered['mo_short_name'].unique()
primer_mo_name = []
for mo_name in primer:
    primer_mo_name.append(mo_name)

In [433]:
df_primer_mo_name  = pd.DataFrame({'mo_short_name': primer_mo_name})

In [434]:
merge_ras_1c = pd.merge(ras1c_filtered, medorganization, on='mo_short_name', how='left')

In [435]:
merge_ras_1c.rename(columns={'mp_dolgnost': 'Должность'}, inplace=True)

In [436]:
merge_ras_1c_uchet = pd.merge(employee_uchet, merge_ras_1c, on=['Организация', 'Должность']).fillna(0)

/tmp/ipykernel_7504/818530909.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merge_ras_1c_uchet = pd.merge(employee_uchet, merge_ras_1c, on=['Организация', 'Должность']).fillna(0)


In [437]:
merge_ras_1c_uchet = merge_ras_1c_uchet[merge_ras_1c_uchet['Проверка'] != 0]

In [438]:
merge_ras_1c_uchet.reset_index(drop=True, inplace=True)

In [439]:
# Отбираем строки в DataFrame, попадающие в заданный период
merge_ras_1c_uchet = merge_ras_1c_uchet[(merge_ras_1c_uchet['date_report'] >= start_date) & (merge_ras_1c_uchet['date_report'] <= end_date)]

In [440]:
merge_ras_1c_uchet.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21577 entries, 0 to 87441
Data columns (total 25 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   Организация                       21577 non-null  object        
 1   Должность                         21577 non-null  object        
 2   Проверка                          21577 non-null  float64       
 3   region                            21577 non-null  object        
 4   mo_oid                            21577 non-null  object        
 5   mo_short_name                     21577 non-null  object        
 6   sp_oid                            21577 non-null  object        
 7   sp_name                           21577 non-null  object        
 8   mp_id                             21577 non-null  int64         
 9   mp_fio                            21577 non-null  object        
 10  date_report                       21577 non-null  d

In [441]:
merge_ras_1c_uchet['date_report'].unique()

<DatetimeArray>
['2024-02-19 00:00:00', '2024-02-20 00:00:00', '2024-02-21 00:00:00',
 '2024-02-22 00:00:00', '2024-02-24 00:00:00', '2024-02-25 00:00:00',
 '2024-02-23 00:00:00']
Length: 7, dtype: datetime64[ns]

##### Расписание из 1С выгружено и обработано

In [442]:
merge_ras_1c_uchet.columns

Index(['Организация', 'Должность', 'Проверка', 'region', 'mo_oid',
       'mo_short_name', 'sp_oid', 'sp_name', 'mp_id', 'mp_fio', 'date_report',
       'mp_stavka', 'slots_type', 'slots_referral', 'slot_lenght',
       'slots_created', 'slots_booked', 'slots_free_after_booked',
       'visits_absence', 'visits_success', 'Наименование МО', 'МО',
       'МедОрганизация', 'Краткое наименование организации',
       'Медицинская организация'],
      dtype='object')

In [443]:
pivot_ras_1c = pd.pivot_table(merge_ras_1c_uchet, index='Организация', columns = 'slots_type', values= 'slots_created', aggfunc=np.sum, margins=True, margins_name='Итого').reset_index()

/tmp/ipykernel_7504/731860570.py:1: FutureWarning: The provided callable <function sum at 0x7eff90181d80> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot_ras_1c = pd.pivot_table(merge_ras_1c_uchet, index='Организация', columns = 'slots_type', values= 'slots_created', aggfunc=np.sum, margins=True, margins_name='Итого').reset_index()
/tmp/ipykernel_7504/731860570.py:1: FutureWarning: The provided callable <function sum at 0x7eff90181d80> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot_ras_1c = pd.pivot_table(merge_ras_1c_uchet, index='Организация', columns = 'slots_type', values= 'slots_created', aggfunc=np.sum, margins=True, margins_name='Итого').reset_index()
/tmp/ipykernel_7504/731860570.py:1: FutureWarning: The provided ca

In [444]:
pivot_ras_1c['Доля конкурентных слотов от общего количества слотов, не менее 65%'] = round(pivot_ras_1c['Доступен'] / (pivot_ras_1c['Доступен'] + pivot_ras_1c['Не доступен']) * 100, 2)
pivot_ras_1c_s2 = pivot_ras_1c[['Организация', 'Доля конкурентных слотов от общего количества слотов, не менее 65%']]
pivot_ras_1c_s2 = pivot_ras_1c_s2.rename_axis(None, axis=1)

4. Доля конкурентных слотов участковой службы от общего количества слотов

In [445]:
ras_1c_uchaskov = merge_ras_1c_uchet[(merge_ras_1c_uchet['Должность'] ==  'врач-педиатр участковый') |  (merge_ras_1c_uchet['Должность'] ==  'врач-терапевт участковый')]
pivot_ras_1c_uchstkov = pd.pivot_table(ras_1c_uchaskov, index='Организация', columns = 'slots_type', values= 'slots_created', aggfunc=np.sum, margins=True, margins_name='Итого').reset_index()
pivot_ras_1c_uchstkov['Доля конкурентных слотов Фактического расписания участковой службы от общего количества слотов, не менее 65%'] = round(pivot_ras_1c_uchstkov['Доступен'] / (pivot_ras_1c_uchstkov['Доступен'] + pivot_ras_1c_uchstkov['Не доступен']) * 100, 2)
pivot_ras_1c_uchstkov.reset_index(drop=True, inplace= True)

/tmp/ipykernel_7504/1135622585.py:2: FutureWarning: The provided callable <function sum at 0x7eff90181d80> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot_ras_1c_uchstkov = pd.pivot_table(ras_1c_uchaskov, index='Организация', columns = 'slots_type', values= 'slots_created', aggfunc=np.sum, margins=True, margins_name='Итого').reset_index()
/tmp/ipykernel_7504/1135622585.py:2: FutureWarning: The provided callable <function sum at 0x7eff90181d80> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot_ras_1c_uchstkov = pd.pivot_table(ras_1c_uchaskov, index='Организация', columns = 'slots_type', values= 'slots_created', aggfunc=np.sum, margins=True, margins_name='Итого').reset_index()
/tmp/ipykernel_7504/1135622585.py:2: FutureWarning: 

In [446]:
pivot_ras_1c_uchstkov_s2 = pivot_ras_1c_uchstkov[['Организация', 'Доля конкурентных слотов Фактического расписания участковой службы от общего количества слотов, не менее 65%']]

In [447]:
pivot_ras_1c_uchstkov_s2 = pivot_ras_1c_uchstkov_s2.rename_axis(None, axis=1)
pivot_ras_1c_uchstkov_s2.head(1)

,Организация,"Доля конкурентных слотов Фактического расписания участковой службы от общего количества слотов, не менее 65%"
0,ГАУЗ ТО «ГП №12»,53.68


* Доля конкурентных слотов ПЛАН составленного расписания, не менее 65%

In [448]:
input_analit_filtr = pd.read_excel(FILTR_ANALIT)

In [449]:
input_analit_filtr = input_analit_filtr[(input_analit_filtr['Медицинская организация'] != 'Всего')]


In [450]:
analit_merge_filtr = pd.merge(input_analit_filtr, medorganization, on='Медицинская организация', how='left')

In [451]:
analit_merge_filtr.columns

Index(['Медицинская организация', 'Отделение', 'Отделение (адрес)',
       'Должность', 'Специальность', 'Врач', 'Дней расписания',
       'Слотов на 7 дней', 'Слотов на 14 дней', 'Слотов на 30 дней',
       'Формиров. расписания на 7 дней, %',
       'Формиров. расписания на 14 дней, %',
       'Формиров. расписания на 30 дней, %',
       'Слотов первичного приёма на 7 дней',
       'Доля первичного приёма на 7 дней, %',
       'Слотов первичного приёма на 14 дней',
       'Доля первичного приёма на 14 дней, %',
       'Слотов первичного приёма на 30 дней',
       'Доля первичного приёма на 30 дней, %', 'Наименование МО',
       'Организация', 'МО', 'МедОрганизация',
       'Краткое наименование организации', 'mo_short_name'],
      dtype='object')

In [452]:
delete_rows3 = pd.read_excel(GUIDE_DELETE_MED, skiprows=6)
delete_rows3.rename(columns={'Unnamed: 0': 'СНИЛС', 
                    'Unnamed: 1': 'Фамилия', 
                    'Unnamed: 2': 'Имя', 
                    'Unnamed: 3': 'Отчество', 
                    'Unnamed: 4': 'Пол', 
                    'Unnamed: 5': 'Дата рождения', 
                    'Unnamed: 6': 'Сведения о трудоустройстве'}, inplace=True)

In [453]:
delete_rows3 = delete_rows3[['Краткое наименование организации', 'Наименование структурного подразделения', 'Должность по фед справочнику']]
delete_rows3 = delete_rows3.rename(columns={'Должность по фед справочнику' : 'Должность', 'Наименование структурного подразделения': 'Отделение'})
delete_rows3['Удалить'] = 'True'

In [454]:
delete_rows3.head(1)

,Краткое наименование организации,Отделение,Должность,Удалить
0,"ГАУЗ ТО ""ГОРОДСКАЯ ПОЛИКЛИНИКА №3""",Отделение организации медицинской помощи студе...,врач-офтальмолог,True


In [455]:
result_original_filtr = pd.merge(analit_merge_filtr, delete_rows3, on=['Краткое наименование организации', 
                                      'Отделение', 
                                      'Должность'], how='left')
result_original_filtr = result_original_filtr[result_original_filtr['Удалить'] != 'True']

##### Подгружаем таблицу должностей и мержим ее с таблицей из Аналит

##### Это таблица должностей аналит

In [456]:
merge_table_analit_result_filter = pd.merge(result_original_filtr, table_analit_doljnost, on = ['Должность', 'Специальность'], how='right')

In [457]:
merge_table_analit_result_filter['Доля первичного приёма на 7 дней, %'] = merge_table_analit_result_filter['Доля первичного приёма на 7 дней, %'].replace('-', 0)

/tmp/ipykernel_7504/2356964169.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merge_table_analit_result_filter['Доля первичного приёма на 7 дней, %'] = merge_table_analit_result_filter['Доля первичного приёма на 7 дней, %'].replace('-', 0)


In [458]:
merge_table_analit_result_filter = merge_table_analit_result_filter[(merge_table_analit_result_filter['Должность'] == 'врач-терапевт участковый') | 
                          (merge_table_analit_result_filter['Должность'] == 'врач-педиатр участковый')]

In [459]:
svod_slotov = pd.pivot_table(df1, values=['Слотов на 7 дней', 
                                          'Слотов первичного приёма на 7 дней'], 
                             index=['Организация'], 
                             aggfunc=np.sum, 
                             fill_value=0, margins=True, margins_name='Итого')

/tmp/ipykernel_7504/764798134.py:1: FutureWarning: The provided callable <function sum at 0x7eff90181d80> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  svod_slotov = pd.pivot_table(df1, values=['Слотов на 7 дней',


In [460]:
svod_slotov.reset_index(drop=False, inplace=True)

In [461]:
def proz7days(row):
    if row['Слотов первичного приёма на 7 дней'] != 0 and row['Слотов на 7 дней'] != 0:
        val = round(row['Слотов первичного приёма на 7 дней'] / row['Слотов на 7 дней'] * 100, 2)
    else:
        val = 0
    return val

In [462]:
svod_slotov['% конкурентных слотов на 7 дней'] = svod_slotov.apply(proz7days, axis = 1)

In [463]:
svod_slotov = svod_slotov[['Организация', '% конкурентных слотов на 7 дней']]

In [464]:
svod_slotov.rename(columns={'% конкурентных слотов на 7 дней': 'Доля конкурентных слотов ПЛАН составленного расписания, не менее 65%'}, inplace=True)

##### Итог расчета слайда 2

In [465]:
data_frames = [itog_slots, pivot_ras_1c_s2, itog_slots_uchastkov, svod_slotov, pivot_ras_1c_uchstkov_s2]

In [466]:
pivot_ras_1c_s2 = reduce(lambda  left,right: pd.merge(left,right,on=['Организация'], how='outer'), data_frames)

In [467]:
result_slide2 = pivot_ras_1c_s2[['Организация', 'Доля фактического количества слотов от расчетных, не менее 80%', 
                                 'Доля конкурентных слотов от общего количества слотов, не менее 65%', 
                                 'Доля фактического количества слотов от расчетных у врачей участковой службы, не менее 80%', 
                                 'Доля конкурентных слотов ПЛАН составленного расписания, не менее 65%',
                                 'Доля конкурентных слотов Фактического расписания участковой службы от общего количества слотов, не менее 65%']]
result_slide2.rename(columns={'Организация': 'Наименование МО'}, inplace=True)
result_slide2.fillna(0, inplace=True)

In [468]:
# Определяем значение, по которому будем перемещать строку в конец
value_to_move = 'Итого'

# Выбираем строку по заданному значению
row_to_move = result_slide2[result_slide2['Наименование МО'] == value_to_move]

In [469]:
# Удаляем эту строку из исходного места
result_slide2 = result_slide2.drop(row_to_move.index)

In [470]:
result_slide2.reset_index(drop=True, inplace=True)

In [471]:
sequence_mo2 = sequence_mo.rename(columns={'Организация': 'Наименование МО'})

In [472]:
sequence_mo2 = sequence_mo2[sequence_mo2['Наименование МО'] != 'Итог']

In [473]:
sequence_mo2

,Наименование МО
0,ГАУЗ ТО «ГП №1»
1,ГАУЗ ТО «ГП №12»
2,ГАУЗ ТО «ГП №13»
3,ГАУЗ ТО «ГП №17»
4,ГАУЗ ТО «ГП №3»
5,ГАУЗ ТО «ГП №4»
6,ГАУЗ ТО «ГП №5»
7,ГАУЗ ТО «ГП №6»
8,ГАУЗ ТО «ГП №8»
9,"ГАУЗ ТО ""ОСП"""


In [474]:
df2 = pd.merge(sequence_mo2, result_slide2, how='left', on='Наименование МО')

In [475]:
# Добавляем эту строку в конец DataFrame
df2 = pd.concat([df2, row_to_move])

In [476]:
df2.reset_index(drop=True, inplace=True)

##### Подготовка данных для слайда 3

1. Расчет первого столбца 'Занятость слотов всего (доля от общего числа созданных и количества занятых)'
* Использую уже сверенный с проверочными файлами результат обработки расписания из 1С

In [477]:
slide_3_column_1 = merge_ras_1c_uchet
slide_3_column_1.rename(columns={'Должность': 'mp_dolgnost'}, inplace=True)

In [478]:
merge_ras_1c_uchet['date_report'].unique()

<DatetimeArray>
['2024-02-19 00:00:00', '2024-02-20 00:00:00', '2024-02-21 00:00:00',
 '2024-02-22 00:00:00', '2024-02-24 00:00:00', '2024-02-25 00:00:00',
 '2024-02-23 00:00:00']
Length: 7, dtype: datetime64[ns]

In [479]:
slide_3_column_1 = slide_3_column_1[['МО', 'mp_dolgnost', 'slots_type', 'slots_created', 'slots_booked']]

In [480]:
slide_3_column_1_pivot = pd.pivot_table(slide_3_column_1, index='МО', 
                                        columns='slots_type', 
                                        values=['slots_created', 'slots_booked'],
                                        aggfunc=np.sum, margins=True, margins_name='Итого').reset_index()

/tmp/ipykernel_7504/108592405.py:1: FutureWarning: The provided callable <function sum at 0x7eff90181d80> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  slide_3_column_1_pivot = pd.pivot_table(slide_3_column_1, index='МО',
/tmp/ipykernel_7504/108592405.py:1: FutureWarning: The provided callable <function sum at 0x7eff90181d80> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  slide_3_column_1_pivot = pd.pivot_table(slide_3_column_1, index='МО',
/tmp/ipykernel_7504/108592405.py:1: FutureWarning: The provided callable <function sum at 0x7eff90181d80> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  slide_3_column_1_

In [481]:
slide_3_column_1_pivot = slide_3_column_1_pivot.droplevel(0, axis=1)

In [482]:
#В этом примере мы создаем DataFrame с неуникальными именами столбцов. Мы используем метод MultiIndex.from_tuples для создания мультииндексов, чтобы сделать имена столбцов уникальными путем добавления индекса к дублирующимся именам. После этого мы присваиваем новые имена столбцов DataFrame.
slide_3_column_1_pivot.columns = pd.MultiIndex.from_tuples([(col, 'slots_type' + str(i)) if col else (col, '') for i, col in enumerate(slide_3_column_1_pivot.columns)])

In [483]:
slide_3_column_1_pivot = slide_3_column_1_pivot.droplevel(0, axis=1)

In [484]:
slide_3_column_1_pivot = slide_3_column_1_pivot.rename(columns={'': 'МО'})
slide_3_column_1_pivot = slide_3_column_1_pivot[['МО', 'slots_type3', 'slots_type6']]
slide_3_column_1_pivot = slide_3_column_1_pivot.rename_axis(None, axis=1)
slide_3_column_1_pivot['Занятость слотов всего (доля от общего числа созданных и количества занятых)'] = round(slide_3_column_1_pivot['slots_type3'] / slide_3_column_1_pivot['slots_type6'] * 100, 2)

In [485]:
slide_3_column_1_itog = slide_3_column_1_pivot[['МО', 'Занятость слотов всего (доля от общего числа созданных и количества занятых)']]

2. 'Доля неявок от общего числа записей'

In [486]:
slide_3_column2 = merge_ras_1c_uchet
slide_3_column2.rename(columns={'Должность': 'mp_dolgnost'}, inplace=True)

In [487]:
slide3_column2 = slide_3_column2[['МО', 'mp_dolgnost', 'slots_type', 'slots_booked', 'visits_absence']]

In [488]:
slide3_column2_pivot = pd.pivot_table(slide3_column2, index='МО', 
                                        columns='slots_type', 
                                        values=['slots_booked', 'visits_absence'],
                                        aggfunc=np.sum, margins=True, margins_name='Итого').reset_index()

/tmp/ipykernel_7504/643692928.py:1: FutureWarning: The provided callable <function sum at 0x7eff90181d80> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  slide3_column2_pivot = pd.pivot_table(slide3_column2, index='МО',
/tmp/ipykernel_7504/643692928.py:1: FutureWarning: The provided callable <function sum at 0x7eff90181d80> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  slide3_column2_pivot = pd.pivot_table(slide3_column2, index='МО',
/tmp/ipykernel_7504/643692928.py:1: FutureWarning: The provided callable <function sum at 0x7eff90181d80> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  slide3_column2_pivot = pd

In [489]:
slide3_column2_pivot = slide3_column2_pivot.droplevel(0, axis=1)

In [490]:
#В этом примере мы создаем DataFrame с неуникальными именами столбцов. Мы используем метод MultiIndex.from_tuples для создания мультииндексов, чтобы сделать имена столбцов уникальными путем добавления индекса к дублирующимся именам. После этого мы присваиваем новые имена столбцов DataF
slide3_column2_pivot.columns = pd.MultiIndex.from_tuples([(col, 'slots_type' + str(i)) if col else (col, '') for i, col in enumerate(slide3_column2_pivot.columns)])

In [491]:
slide3_column2_pivot = slide3_column2_pivot.droplevel(0, axis=1)

In [492]:
slide3_column2_pivot = slide3_column2_pivot.rename(columns={'': 'МО'})
slide3_column2_pivot = slide3_column2_pivot[['МО', 'slots_type4', 'slots_type3']]
slide3_column2_pivot = slide3_column2_pivot.rename_axis(None, axis=1)
slide3_column2_pivot['Доля неявок от общего числа записей'] = round(slide3_column2_pivot['slots_type4'] / slide3_column2_pivot['slots_type3'] * 100, 2)

In [493]:
slide3_column2_itog = slide3_column2_pivot[['МО', 'Доля неявок от общего числа записей']]

3. Расчет третьего столбца 'Доля пациентов, принятых без предварительной записи'

In [494]:
indi.columns

Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Общее число слотов в расписании', 'из них конкурентных слотов',
       'Общее число записей', 'из них вне слота с признаком "живая очередь"',
       'из них 2-я/3-я/4-я запись пациентов на один и тот же слот времени',
       'Общее число записей в расписании (занятые слоты)', 'из них из Прочие',
       'из них из ЕПГУ', 'из них из региональный портал',
       'из них из единого колл-центра',
       'из них из колл-центров медицинских организаций',
       'из них из регистратуры', 'из них из инфомата',
       'Число отмененных записей',
       'Число поступивших обращений с информационной целью (запрос информации)',
       'Число поступивших обращений с целью записи на прием к врачу-специалисту по поводу заболеваний и с профилактической целью первичная',
       'Число поступивших обращений с целью записи на прием к врачу-специалисту по поводу заболеваний повторная',
       'Число поступивших обращений с

In [495]:
indicators_slide3 = indi.copy()

In [496]:
indicators_slide3.rename(columns={'Unnamed: 0': 'МО', 'Unnamed: 3': 'Должность'}, inplace=True)

In [497]:
indicators_slide3.rename(columns={'МО': 'Медицинская организация'}, inplace=True)

In [498]:
indicators_slide3.columns

Index(['Медицинская организация', 'Unnamed: 1', 'Unnamed: 2', 'Должность',
       'Unnamed: 4', 'Общее число слотов в расписании',
       'из них конкурентных слотов', 'Общее число записей',
       'из них вне слота с признаком "живая очередь"',
       'из них 2-я/3-я/4-я запись пациентов на один и тот же слот времени',
       'Общее число записей в расписании (занятые слоты)', 'из них из Прочие',
       'из них из ЕПГУ', 'из них из региональный портал',
       'из них из единого колл-центра',
       'из них из колл-центров медицинских организаций',
       'из них из регистратуры', 'из них из инфомата',
       'Число отмененных записей',
       'Число поступивших обращений с информационной целью (запрос информации)',
       'Число поступивших обращений с целью записи на прием к врачу-специалисту по поводу заболеваний и с профилактической целью первичная',
       'Число поступивших обращений с целью записи на прием к врачу-специалисту по поводу заболеваний повторная',
       'Число пост

In [499]:
indicators_slide3 = indicators_slide3[['Медицинская организация', 
                    'Должность', 
                    'Общее число слотов в расписании', 
                    'Общее число записей', 
                    'из них вне слота с признаком "живая очередь"', 
                    'из них 2-я/3-я/4-я запись пациентов на один и тот же слот времени', 
                    'Общее число записей в расписании (занятые слоты)', 'из них из Прочие']].fillna(0)

In [500]:
indicators_slide3.columns

Index(['Медицинская организация', 'Должность',
       'Общее число слотов в расписании', 'Общее число записей',
       'из них вне слота с признаком "живая очередь"',
       'из них 2-я/3-я/4-я запись пациентов на один и тот же слот времени',
       'Общее число записей в расписании (занятые слоты)', 'из них из Прочие'],
      dtype='object')

In [501]:
indicators2 =  pd.merge(indicators_slide3, medorganization, how='left', on='Медицинская организация').dropna()

In [502]:
indicators2 = pd.merge(tablica_doljnost, indicators2, how='left', on='Должность').dropna()

In [503]:
# сопоставление 
indicator_mo_uchet_slide3 = pd.merge(indicators2, employee_uchet, on=['Организация', 'Должность'], how='left').fillna(0)

In [504]:

slide3_column3_svod = pd.pivot_table(indicator_mo_uchet_slide3, index='МО', values=['Общее число слотов в расписании', 
                                                                      'Общее число записей', 
                                                                      'из них вне слота с признаком "живая очередь"', 
                                                                      'из них 2-я/3-я/4-я запись пациентов на один и тот же слот времени', 
                                                                      'Общее число записей в расписании (занятые слоты)'], 
                                                                      aggfunc=np.sum, margins=True, margins_name='Итого').reset_index()
slide3_column3_svod['Доля пациентов, принятых без предварительной записи'] = round(slide3_column3_svod['из них вне слота с признаком "живая очередь"'] / slide3_column3_svod['Общее число записей'] * 100, 2).fillna(0)
slide3_column3_svod = slide3_column3_svod[slide3_column3_svod['МО'] != 'ПЦ_Тюмень']
slide3_column3_svod = slide3_column3_svod[slide3_column3_svod['МО'] != 'ОКБ1']
slide3_column3 = slide3_column3_svod[['МО', 'Доля пациентов, принятых без предварительной записи']]

/tmp/ipykernel_7504/3860048293.py:1: FutureWarning: The provided callable <function sum at 0x7eff90181d80> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  slide3_column3_svod = pd.pivot_table(indicator_mo_uchet_slide3, index='МО', values=['Общее число слотов в расписании',


4. Расчет показателя 'Доля посещений на дому'

In [505]:
def get_row_group_level(worksheet, row_num):
    cell = worksheet.cell(row=row_num, column=1)
    indentation_level = cell.alignment.indent
    return indentation_level

def count_row_group_levels(file_path):
    workbook = openpyxl.load_workbook(file_path)
    worksheet = workbook.active

    max_row = worksheet.max_row
    group_levels = {}

    for row_num in range(8, max_row + 1):
        group_level = get_row_group_level(worksheet, row_num)
        group_levels[row_num] = group_level

    return group_levels

folder_path = EMERGENCY_XLSX
combined_group_levels = []

for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx'):
        file_path = os.path.join(folder_path, filename)
        row_group_levels = count_row_group_levels(file_path)
        dff = pd.DataFrame.from_dict(row_group_levels, orient='index', columns=['Уровень структуры'])
        dff['Файл'] = filename
        combined_group_levels.append(dff)

combined_df = pd.concat(combined_group_levels)
combined_df.index.name = 'Строка'
combined_df = combined_df.reset_index()
combined_df['Файл'] = combined_df['Файл'].apply(lambda x: x.split('.')[0])
clean_df = combined_df[['Уровень структуры']]
clean_df.loc[clean_df['Уровень структуры'] == 0, 'Уровень структуры'] = 1
clean_df

/tmp/ipykernel_7504/598535920.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df.loc[clean_df['Уровень структуры'] == 0, 'Уровень структуры'] = 1


,Уровень структуры
0,1.0
1,2.0
2,2.0
3,2.0
4,2.0
...,...
3184,2.0
3185,2.0
3186,2.0
3187,2.0


In [506]:
NEOTLOJKA_XLSX

'/home/serg/prezentation/emergency/neotlojka.xlsx'

In [507]:
emergency_medical_care = pd.read_excel(NEOTLOJKA_XLSX, skiprows=[0, 1, 2, 3, 4]).fillna(0)

In [508]:
emergency_medical_care

,Должность,Unnamed: 1,Unnamed: 2,Источник записи строка,Unnamed: 4,Всего количество записей на прием,"Записи на тип приема врача ""Вызов на дом""",Unnamed: 7,Unnamed: 8,"Метка ""Помощь на дому""","Метка ""Неотложная помощь в поликлинике"""
0,0,0.0,0.0,0,0.0,0.0,"Метка ""Неотложная помощь на дому""","Метка ""Вызов""","Метка ""Патронаж новорожденного""",0.0,0.0
1,"ГАУЗ ТО ""Городская поликлиника № 1""",0.0,0.0,0,0.0,0.0,0,0,0,0.0,0.0
2,врач общей практики (семейный врач),0.0,0.0,Запись на прием с ЕПГУ,0.0,8.0,0,0,0,0.0,0.0
3,врач-акушер-гинеколог,0.0,0.0,Запись на прием с ЕПГУ,0.0,30.0,0,0,0,0.0,0.0
4,врач-оториноларинголог,0.0,0.0,Запись на прием с ЕПГУ,0.0,6.0,0,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
3185,акушер,0.0,0.0,АРМ медицинского работника,0.0,70.0,0,0,0,0.0,0.0
3186,медицинская сестра процедурной,0.0,0.0,АРМ медицинского работника,0.0,3.0,0,0,0,0.0,0.0
3187,фельдшер,0.0,0.0,АРМ медицинского работника,0.0,38.0,0,0,0,14.0,0.0
3188,врач-кардиолог,0.0,0.0,АРМ медицинского работника,0.0,37.0,0,0,0,0.0,0.0


In [509]:
emergency_medical_care = emergency_medical_care.rename(columns={'Записи на тип приема врача "Вызов на дом"': 'Метка "Неотложная помощь на дому"', 'Должность': 'Медицинская организация'})
emergency_medical_care.drop(index=0, inplace=True)

In [510]:
emergency_medical_care.reset_index(drop=True, inplace=True)

In [511]:
emergency_medical_care = emergency_medical_care[['Медицинская организация', 'Метка "Неотложная помощь на дому"', 'Метка "Неотложная помощь в поликлинике"']]

In [512]:
emergency_medical_care_merge = pd.concat([emergency_medical_care, clean_df], axis=1)

In [513]:
emergency_medical_care_merge['МО'] = np.where(emergency_medical_care_merge['Уровень структуры'] == 1, emergency_medical_care_merge['Медицинская организация'], np.NaN)
emergency_medical_care_merge['МО'] = emergency_medical_care_merge['МО'].ffill()

In [514]:
emergency_medical_care_merge = emergency_medical_care_merge[emergency_medical_care_merge['Уровень структуры'] != 1.0]

In [515]:
emergency_medical_care_merge = emergency_medical_care_merge[(emergency_medical_care_merge['Медицинская организация'] == 'врач-педиатр участковый') | (emergency_medical_care_merge['Медицинская организация'] == 'врач-терапевт участковый')]

In [516]:
emergency_medical_care_merge = emergency_medical_care_merge[['МО', 'Метка "Неотложная помощь на дому"', 'Метка "Неотложная помощь в поликлинике"']]

In [517]:
emergency_medical_care_merge.to_excel('/home/serg/prezentation/output/neotlojka.xlsx', index=False)

In [518]:
emergency_medical_care_pivot1 = pd.pivot_table(emergency_medical_care_merge, index='МО', values=['Метка "Неотложная помощь на дому"'], aggfunc=np.sum).reset_index()
emergency_medical_care_pivot2 = pd.pivot_table(emergency_medical_care_merge, index='МО', values=['Метка "Неотложная помощь в поликлинике"'], aggfunc=np.sum).reset_index()
emergency_medical_care_pivot_merge2 = pd.merge(emergency_medical_care_pivot1, emergency_medical_care_pivot2, how='inner', on='МО')
emergency_medical_care_pivot_merge2['Сумма посещений на дому  по неотложной помощи у врачей участковой службы'] = emergency_medical_care_pivot_merge2['Метка "Неотложная помощь на дому"'] + emergency_medical_care_pivot_merge2['Метка "Неотложная помощь в поликлинике"']
emergency_medical_care_pivot_merge2.rename(columns={'МО' : 'mo_short_name'}, inplace=True)

/tmp/ipykernel_7504/45247297.py:1: FutureWarning: The provided callable <function sum at 0x7eff90181d80> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  emergency_medical_care_pivot1 = pd.pivot_table(emergency_medical_care_merge, index='МО', values=['Метка "Неотложная помощь на дому"'], aggfunc=np.sum).reset_index()
/tmp/ipykernel_7504/45247297.py:2: FutureWarning: The provided callable <function sum at 0x7eff90181d80> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  emergency_medical_care_pivot2 = pd.pivot_table(emergency_medical_care_merge, index='МО', values=['Метка "Неотложная помощь в поликлинике"'], aggfunc=np.sum).reset_index()


###### Загружаем секретный отчет

In [519]:
merge_ras_1c_uchet.columns

Index(['Организация', 'mp_dolgnost', 'Проверка', 'region', 'mo_oid',
       'mo_short_name', 'sp_oid', 'sp_name', 'mp_id', 'mp_fio', 'date_report',
       'mp_stavka', 'slots_type', 'slots_referral', 'slot_lenght',
       'slots_created', 'slots_booked', 'slots_free_after_booked',
       'visits_absence', 'visits_success', 'Наименование МО', 'МО',
       'МедОрганизация', 'Краткое наименование организации',
       'Медицинская организация'],
      dtype='object')

In [520]:
merge_ras_1c_uchet_secret = merge_ras_1c_uchet.copy()

In [521]:
merge_ras_1c_uchet_secret.rename(columns={'mp_dolgnost' : 'Должность'}, inplace=True)

In [522]:
merge_ras_1c_uchet_secret['date_report'].unique()

<DatetimeArray>
['2024-02-19 00:00:00', '2024-02-20 00:00:00', '2024-02-21 00:00:00',
 '2024-02-22 00:00:00', '2024-02-24 00:00:00', '2024-02-25 00:00:00',
 '2024-02-23 00:00:00']
Length: 7, dtype: datetime64[ns]

In [523]:
uchastkovye_secret = merge_ras_1c_uchet_secret[(merge_ras_1c_uchet_secret['Должность'] == 'врач-педиатр участковый') | (merge_ras_1c_uchet_secret['Должность'] == 'врач-терапевт участковый')]

In [524]:
uchastkovye_secret['Должность'].unique()

array(['врач-педиатр участковый', 'врач-терапевт участковый'],
      dtype=object)

In [525]:
uchastkovye_secret['count_number'] = 1
uchastkovye_secret_pivot = pd.pivot_table(uchastkovye_secret, index=['МО'], columns='Должность', values='count_number', aggfunc=np.sum, margins=True, margins_name='Итого').reset_index()
uchastkovye_secret_pivot.rename_axis(None, axis=1, inplace=True)

/tmp/ipykernel_7504/1098382493.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uchastkovye_secret['count_number'] = 1
/tmp/ipykernel_7504/1098382493.py:2: FutureWarning: The provided callable <function sum at 0x7eff90181d80> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  uchastkovye_secret_pivot = pd.pivot_table(uchastkovye_secret, index=['МО'], columns='Должность', values='count_number', aggfunc=np.sum, margins=True, margins_name='Итого').reset_index()
/tmp/ipykernel_7504/1098382493.py:2: FutureWarning: The provided callable <function sum at 0x7eff90181d80> is currently using DataFrameGroupBy.sum. In a future versio

In [526]:

uchastkovye_secret_pivot['Участковые'] = uchastkovye_secret_pivot['врач-педиатр участковый'] + uchastkovye_secret_pivot['врач-терапевт участковый']

In [527]:
uchastkovye_itog = uchastkovye_secret_pivot[['МО', 'Участковые']]

In [528]:
uchastkovye_itog['МО'].unique()

array(['ГП1', 'ГП12', 'ГП13', 'ГП17', 'ГП3', 'ГП4', 'ГП5', 'ГП6', 'ГП8',
       'ОБ11', 'ОБ12', 'ОБ13', 'ОБ14', 'ОБ15', 'ОБ19', 'ОБ20', 'ОБ23',
       'ОБ24', 'ОБ3', 'ОБ4', 'ОБ9', 'ОКБ2', 'Итого'], dtype=object)

In [529]:
guide_mo2 = pd.read_excel(ROOT_PATH +'/guide/guide_mo2.xlsx')

In [530]:
guide_mo2

,mo_short_name,МО
0,"ГАУЗ ТО ""Городская поликлиника № 1""",ГП1
1,"ГАУЗ ТО ""Городская поликлиника № 12""",ГП12
2,"ГАУЗ ТО ""Городская поликлиника № 13 """,ГП13
3,"ГАУЗ ТО ""Городская поликлиника № 4""",ГП4
4,"ГАУЗ ТО ""Городская поликлиника №5""",ГП5
5,"ГАУЗ ТО ""Областная больница № 19"" (г.Тюмень)",ОБ19
6,ГАУЗ ТО «Городская Поликлиника № 6»,ГП6
7,ГАУЗ ТО «Городская поликлиника № 8»,ГП8
8,ГАУЗ ТО «Городская поликлиника №17»,ГП17
9,ГАУЗ ТО «Городская поликлиника №3»,ГП3


In [531]:
emergency_medical_guide = pd.merge(guide_mo2, emergency_medical_care_pivot_merge2, on='mo_short_name', how='right')

###### Объединяеем план и факт расчета по участковой службе

In [532]:
emergency_medical_guide_itog = emergency_medical_guide[['МО', 'Сумма посещений на дому  по неотложной помощи у врачей участковой службы']]

In [533]:
emergency_medical_guide_itog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 2 columns):
 #   Column                                                                    Non-Null Count  Dtype 
---  ------                                                                    --------------  ----- 
 0   МО                                                                        20 non-null     object
 1   Сумма посещений на дому  по неотложной помощи у врачей участковой службы  21 non-null     object
dtypes: object(2)
memory usage: 464.0+ bytes


In [534]:
itog_columns = 'Итого'

In [535]:
sum_columns = emergency_medical_guide_itog['Сумма посещений на дому  по неотложной помощи у врачей участковой службы'].sum()

In [536]:
sum_columns

1394.0

In [537]:
new_row = {'МО': itog_columns, 'Сумма посещений на дому  по неотложной помощи у врачей участковой службы': sum_columns}

In [538]:
# emergency_medical_guide_itog = emergency_medical_guide_itog.append(new_row, ignore_index=True)
emergency_medical_guide_itog = pd.concat([emergency_medical_guide_itog, pd.DataFrame([new_row])], ignore_index=True)

In [539]:
merge_emergency_uchastkov = pd.merge(emergency_medical_guide_itog, uchastkovye_itog, on='МО', how='outer')

In [540]:
merge_emergency_uchastkov['Доля посещений на дому'] = np.where(merge_emergency_uchastkov['Участковые'] != 0 & (merge_emergency_uchastkov['Сумма посещений на дому  по неотложной помощи у врачей участковой службы'] != 0), round(merge_emergency_uchastkov['Сумма посещений на дому  по неотложной помощи у врачей участковой службы'] / merge_emergency_uchastkov['Участковые'] * 100, 1), 0)

In [541]:
slide_3_column_4_itog = merge_emergency_uchastkov[['МО', 'Доля посещений на дому']]

In [542]:
med_slide3 = medorganization[['МО', 'Организация']]

In [543]:
row_to_move2 = med_slide3[med_slide3['МО'] == 'ОКБ1']

In [544]:
row_to_move2

,МО,Организация
28,ОКБ1,ГБУЗ ТО «ОКБ №1»


In [545]:
med_slide3 = med_slide3.drop(row_to_move2.index)

In [546]:
row_to_move3 = med_slide3[med_slide3['МО'] == 'ПЦ_Тюмень']

In [547]:
med_slide3 = med_slide3.drop(row_to_move3.index)

In [548]:
med_slide3

,МО,Организация
0,ОСП,"ГАУЗ ТО ""ОСП"""
1,ГП1,ГАУЗ ТО «ГП №1»
2,ГП12,ГАУЗ ТО «ГП №12»
3,ГП13,ГАУЗ ТО «ГП №13»
4,ГП17,ГАУЗ ТО «ГП №17»
5,ГП3,ГАУЗ ТО «ГП №3»
6,ГП4,ГАУЗ ТО «ГП №4»
7,ГП6,ГАУЗ ТО «ГП №6»
8,ГП5,ГАУЗ ТО «ГП №5»
9,ГП8,ГАУЗ ТО «ГП №8»


In [549]:
# med_slide3 = med_slide3.append({'МО': 'Итого', 'Организация': 'Итого'}, ignore_index=True)
med_slide3 = pd.concat([med_slide3, pd.DataFrame([{'МО': 'Итого', 'Организация': 'Итого'}])], ignore_index=True)

In [550]:
data_frames3 = [slide_3_column_1_itog, slide3_column2_itog, slide3_column3, slide_3_column_4_itog]

In [551]:
pivot_ras_1c_s3 = reduce(lambda  left,right: pd.merge(left,right,on=['МО'], how='outer'), data_frames3)

In [552]:
pivot_ras_1c_s3 = pivot_ras_1c_s3.dropna(subset=['МО'])

In [553]:
result_slide3 = pd.merge(pivot_ras_1c_s3, med_slide3, how='left', on='МО')

In [554]:
result_slide3 = result_slide3[['Организация', 'Занятость слотов всего (доля от общего числа созданных и количества занятых)', 'Доля неявок от общего числа записей', 'Доля пациентов, принятых без предварительной записи', 'Доля посещений на дому']]

In [555]:
# Создаем DataFrame
result_slide3.rename(columns={'Организация': 'Наименование МО'}, inplace=True)
result_slide3.fillna(0, inplace=True)
df3 = result_slide3

/tmp/ipykernel_7504/966458667.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  result_slide3.fillna(0, inplace=True)


##### Создаем слайд 4

1. Слайд 4 Таблица 1 Столбец 1

In [556]:
# Подгружаем записи
record = indi

In [557]:
record.rename(columns={'Unnamed: 0': 'Медицинская организация', 'Unnamed: 3': 'Должность'}, inplace=True)

In [558]:
record

,Медицинская организация,Unnamed: 1,Unnamed: 2,Должность,Unnamed: 4,Общее число слотов в расписании,из них конкурентных слотов,Общее число записей,"из них вне слота с признаком ""живая очередь""",из них 2-я/3-я/4-я запись пациентов на один и тот же слот времени,...,Число поступивших обращений с целью записи на прием к врачу-специалисту по поводу заболеваний повторная,Число поступивших обращений с целью записи на прием к врачу-специалисту в рамках диспансерного наблюдения,Число поступивших обращений с целью записи на услуги инструментальной и/или лабораторной диагностики,"Число поступивших обращений с целью получения назначения, направления, справки, выписки, листа нетрудоспособности, иных документов",Число поступивших обращений с целью записи для прохождения первого или второго этапа диспансеризации,Число поступивших обращений с целью записи на вакцинацию,Число операторов колл-центров,Число лиц включенных в лист ожидания,Число лиц исключенных из листа ожидания по причине записи,Число звонков для обратной связи (уточнение цели обращения и актуализация записи)
0,"АСУСОНТО ""Щучинский психоневрологический интер...",NaN,NaN,врач-терапевт,NaN,NaN,NaN,27.0,27.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"АСУСОНТО ""Щучинский психоневрологический интер...",NaN,NaN,фельдшер,NaN,NaN,NaN,62.0,62.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ГАУЗ ТО «ГП № 1»,NaN,NaN,врач - детский хирург,NaN,5.0,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,4.0,NaN
3,ГАУЗ ТО «ГП № 1»,NaN,NaN,врач общей практики (семейный врач),NaN,138.0,77.0,174.0,NaN,61.0,...,88.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN
4,ГАУЗ ТО «ГП № 1»,NaN,NaN,врач по медицинской профилактике,NaN,794.0,NaN,706.0,9.0,87.0,...,163.0,81.0,NaN,NaN,50.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,Тюменская больница ФГБУЗ ЗСМЦ ФМБА,NaN,NaN,врач-терапевт,NaN,292.0,NaN,116.0,11.0,NaN,...,NaN,NaN,NaN,63.0,NaN,NaN,NaN,NaN,NaN,NaN
425,Тюменская больница ФГБУЗ ЗСМЦ ФМБА,NaN,NaN,врач-уролог,NaN,34.0,20.0,103.0,63.0,7.0,...,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
426,Тюменская больница ФГБУЗ ЗСМЦ ФМБА,NaN,NaN,врач-хирург,NaN,149.0,54.0,127.0,2.0,28.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
427,Тюменская больница ФГБУЗ ЗСМЦ ФМБА,NaN,NaN,фельдшер,NaN,487.0,NaN,104.0,23.0,NaN,...,NaN,NaN,NaN,52.0,NaN,8.0,NaN,NaN,NaN,NaN


In [559]:
record_merge = pd.merge(medorganization, record, on='Медицинская организация', how='left')

In [560]:
tablica_doljnost

,Должность
0,врач-терапевт участковый
1,врач-педиатр участковый
2,врач общей практики (семейный врач)
3,врач-офтальмолог
4,врач-оториноларинголог
5,врач-хирург
6,врач-акушер-гинеколог
7,врач-психиатр-нарколог
8,врач-фтизиатр участковый
9,врач-фтизиатр


In [561]:
record_merge['Должность'].unique()

array(['врач-стоматолог', 'врач-стоматолог-терапевт',
       'врач-стоматолог-хирург', 'зубной врач', 'врач - детский хирург',
       'врач общей практики (семейный врач)',
       'врач по медицинской профилактике', 'врач-акушер-гинеколог',
       'врач-оториноларинголог', 'врач-офтальмолог', 'врач-педиатр',
       'врач-педиатр участковый', 'врач-терапевт',
       'врач-терапевт участковый', 'врач-уролог', 'врач-хирург',
       'фельдшер', nan, 'врач-психиатр детский',
       'врач-стоматолог детский', 'врач-психиатр-нарколог',
       'врач-фтизиатр', 'врач-психиатр детский участковый',
       'врач-фтизиатр участковый'], dtype=object)

In [562]:
record_merge_doljnost = pd.merge(tablica_doljnost, record_merge, how='left', on='Должность')

In [563]:
record_merge_doljnost['Должность'].unique()

array(['врач-терапевт участковый', 'врач-педиатр участковый',
       'врач общей практики (семейный врач)', 'врач-офтальмолог',
       'врач-оториноларинголог', 'врач-хирург', 'врач-акушер-гинеколог',
       'врач-психиатр-нарколог', 'врач-фтизиатр участковый',
       'врач-фтизиатр', 'врач-стоматолог', 'врач-стоматолог-терапевт',
       'врач-стоматолог детский', 'врач-психиатр детский',
       'врач - детский хирург', 'врач-психиатр детский участковый'],
      dtype=object)

In [564]:
record_merge_uchet = pd.merge(record_merge_doljnost, employee_uchet, on=['Организация', 'Должность'], how='left').fillna(0)

In [565]:
include_to_list = pd.pivot_table(record_merge_uchet, index='МО', values='Число лиц включенных в лист ожидания', aggfunc=np.sum, margins=True, margins_name='Итого').reset_index()

/tmp/ipykernel_7504/4107631880.py:1: FutureWarning: The provided callable <function sum at 0x7eff90181d80> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  include_to_list = pd.pivot_table(record_merge_uchet, index='МО', values='Число лиц включенных в лист ожидания', aggfunc=np.sum, margins=True, margins_name='Итого').reset_index()


2. Слайд 4 Таблица 1 Столбец 2

In [566]:
recorded = pd.pivot_table(record_merge_uchet, index='МО', values='Число лиц исключенных из листа ожидания по причине записи', aggfunc=np.sum, margins=True, margins_name='Итого').reset_index()

/tmp/ipykernel_7504/43731032.py:1: FutureWarning: The provided callable <function sum at 0x7eff90181d80> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  recorded = pd.pivot_table(record_merge_uchet, index='МО', values='Число лиц исключенных из листа ожидания по причине записи', aggfunc=np.sum, margins=True, margins_name='Итого').reset_index()


3. Слайд 4 Таблица 1 Столбец 3

In [567]:
competitive = pd.pivot_table(record_merge_uchet, index='МО', values='из них конкурентных слотов', aggfunc=np.sum, margins=True, margins_name='Итого').reset_index()
competitive.rename(columns={'из них конкурентных слотов' : 'Количество конкурентной записи'}, inplace=True)

/tmp/ipykernel_7504/3296361143.py:1: FutureWarning: The provided callable <function sum at 0x7eff90181d80> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  competitive = pd.pivot_table(record_merge_uchet, index='МО', values='из них конкурентных слотов', aggfunc=np.sum, margins=True, margins_name='Итого').reset_index()


In [568]:
data_frames4 = [include_to_list, recorded, competitive]
slide4_df = reduce(lambda  left,right: pd.merge(left,right,on=['МО'], how='outer'), data_frames4)

###### Слайд 4 столбец 4

In [569]:
slide4_df['Доля лиц, включенных в лист ожидания от общего количества конкурентной записи'] = round((slide4_df['Число лиц включенных в лист ожидания'] / slide4_df['Количество конкурентной записи']) * 100, 2).fillna(0)
slide4_df['Движение лиц в листе ожидания'] = round((slide4_df['Число лиц включенных в лист ожидания'] - slide4_df['Число лиц исключенных из листа ожидания по причине записи']) / slide4_df['Число лиц включенных в лист ожидания'] * 100, 2).fillna(0)

###### Слайд 4 столбец 5

In [570]:
slide4_df.sort_values(by='Движение лиц в листе ожидания', ascending=False, inplace=True)

In [571]:
slide4_df.reset_index(drop=True, inplace=True)

In [572]:
# Получаем индекс строки, которую нужно переместить в конец
index_itog_slide4 = slide4_df.index[slide4_df['МО'] == 'Итого'].tolist()[0]
row_to_move = slide4_df.loc[index_itog_slide4]
slide4_df = slide4_df.drop(index_itog_slide4)
# Добавляем извлеченную строку в конец DataFrame
# slide4_df = slide4_df.append(row_to_move)
slide4_df = pd.concat([slide4_df, pd.DataFrame([row_to_move])], ignore_index=True)

In [573]:
slide4_df.reset_index(drop=True, inplace=True)

In [574]:
# Создаем DataFrame
result_slide4 = pd.merge(slide4_df, med_slide3, how='left', on='МО')
result_slide4 = result_slide4[['Организация', 'Число лиц включенных в лист ожидания', 'Число лиц исключенных из листа ожидания по причине записи', 'Количество конкурентной записи', 'Доля лиц, включенных в лист ожидания от общего количества конкурентной записи', 'Движение лиц в листе ожидания']]
result_slide4.rename(columns={'Организация': 'Наименование МО'}, inplace=True)
result_slide4.fillna(0, inplace=True)
df4 = result_slide4

##### Слайд 5
* Обрабатываем выгрузки из Аналит
* Требуется смотреть по адресу 'h:/СЦ/Исп_показателей_субъекта_Доступность_РЭМД/Сбор данных 2023/Подготовка к ВКС 26.12.2023/И38/для ПРОПП/Аналит доступность'

In [575]:
# Получаем список входных файлов
slide5_input = glob.glob(os.path.join(ANALIT_SLIDE5, "*.xlsx"))

In [576]:
# Объединяем их в один датафрейм
svod_slide5 = []
for my_files in slide5_input:
    next_files = pd.read_excel(my_files, usecols=['Медицинская организация', 'Должность', 'Специальность',  'Врач','Кол.дней ожидания']).dropna()
    svod_slide5.append(next_files)
read_slide5 = pd.concat(svod_slide5, sort=False, axis=0)


###### в системе Аналит для ОБ13 и Перинатального центра есть несколько вариантов наименований. Это может серьезно сказаться на качестве данных. Поэтому я обрабатываю эти наименовния и привожу их к одному варианту.

In [577]:
read_slide5['Медицинская организация'] = np.where(read_slide5['Медицинская организация'] == 'Государственное бюджетное учреждение здравоохранения Тюменской Области Областная больница №13 (с. Исетское)', 
'Государственное бюджетное учреждение здравоохранения Тюменской области Областная больница № 13 (с.Исетское)',
 read_slide5['Медицинская организация'])

In [578]:
read_slide5['Медицинская организация'] = np.where(read_slide5['Медицинская организация'] == 'Государственное бюджетное учреждение здравоохранения Тюменской области "Областная больница № 13 (с.Исетское)"', 
'Государственное бюджетное учреждение здравоохранения Тюменской области Областная больница № 13 (с.Исетское)',
 read_slide5['Медицинская организация'])

In [579]:
read_slide5['Медицинская организация'] = np.where(read_slide5['Медицинская организация'] == 'Государственное бюджетное учреждение здравоохранения Тюменской области "Перинатальный центр" г. Тюмень', 
'Государственное бюджетное учреждение здравоохранения Тюменской области "Перинатальный центр" (г. Тюмень)',
 read_slide5['Медицинская организация'])

In [580]:
read_slide5['Медицинская организация'].unique()

array(['ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХРАНЕНИЯ ТЮМЕНСКОЙ ОБЛАСТИ "ОБЛАСТНАЯ БОЛЬНИЦА №4" (Г.ИШИМ)',
       'Государственное бюджетное учреждение здравоохранения Тюменской области "Областная больница №11" (р.п. Голышманово)',
       'Государственное бюджетное учреждение здравоохранения Тюменской области "Областная больница № 9" (с. Вагай)',
       'ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХРАНЕНИЯ ТЮМЕНСКОЙ ОБЛАСТИ "ОБЛАСТНАЯ БОЛЬНИЦА № 3"',
       'Государственное автономное учреждение здравоохранения Тюменской области «Городская поликлиника № 1»',
       'Государственное автономное учреждение здравоохранения Тюменской области «Городская поликлиника № 3»',
       'Государственное автономное учреждение здравоохранения Тюменской области «Городская поликлиника № 4»',
       'Государственное автономное учреждение здравоохранения Тюменской области «Городская поликлиника № 5»',
       'Государственное автономное учреждение здравоохранения Тюменской области «Городская поликли

In [581]:
read_slide5 = pd.merge(read_slide5, table_analit_doljnost, on=['Должность', 'Специальность'], how='right').dropna()

In [582]:
read_slide5['Кол.дней ожидания']

0                        0.25
1                           1
2                         0.6
3        нет доступной записи
4                           0
                 ...         
48139                       1
48140                     3.2
48141                       3
48142                       2
48143                       1
Name: Кол.дней ожидания, Length: 48143, dtype: object

In [583]:
read_slide5 = read_slide5[['Медицинская организация', 'Должность', 'Врач', 'Кол.дней ожидания']]

In [584]:
# Очистка ячейки, замена на None строковых значений
read_slide5['Кол.дней ожидания'] = np.where(read_slide5['Кол.дней ожидания'] == 'нет доступной записи', None, read_slide5['Кол.дней ожидания'])

In [585]:
# Округляю до наименьшего целого если значение не пусто
read_slide5['Минимум дней'] = read_slide5['Кол.дней ожидания'].map(lambda x: int(math.floor(float(x))) if x is not None else None)

In [586]:
read_slide5['Должность'].unique()

array(['врач-терапевт участковый', 'врач-педиатр участковый',
       'врач общей практики (семейный врач)', 'врач-офтальмолог',
       'врач-оториноларинголог', 'врач-хирург', 'врач-акушер-гинеколог',
       'врач-психиатр-нарколог', 'врач-фтизиатр участковый',
       'врач-фтизиатр', 'врач-стоматолог', 'врач-стоматолог-терапевт',
       'врач-стоматолог детский', 'врач-психиатр детский',
       'врач - детский хирург', 'врач-психиатр детский участковый',
       'врач-психиатр'], dtype=object)

In [587]:
read_slide5['Минимум дней']

0        0.0
1        1.0
2        0.0
3        NaN
4        0.0
        ... 
48139    1.0
48140    3.0
48141    3.0
48142    2.0
48143    1.0
Name: Минимум дней, Length: 48143, dtype: float64

In [588]:
read_slide5.rename(columns={'Медицинская организация': 'Аналит МО'}, inplace=True)

In [589]:
analit_mo = pd.read_excel(ROOT_PATH + '//guide//medicine_organization_3.xlsx')

In [590]:
analit_slide_5 = pd.merge(read_slide5, analit_mo, on='Аналит МО', how='left')

* Этот код фильтрует DataFrame `analit_slide_5` и оставляет только те строки, где значения столбца 'Должность' находятся в списке `employee_list`.

* Специфически, `analit_slide_5['Должность'].isin(employee_list)` создает булеву серию, которая проверяет, находятся ли значения столбца 'Должность' в списке `employee_list`. Затем, `analit_slide_5[...]` используется для * индексации DataFrame, чтобы оставить только строки, соответствующие `True` значениям в булевой серии.

* Таким образом, после выполнения этого кода, `analit_slide_5` будет содержать только строки, где столбец 'Должность' содержит значения из `employee_list`.


In [591]:
analit_slide_5 = analit_slide_5[analit_slide_5['Должность'].isin(employee_list)]

In [592]:
analit_slide_5['Должность'].unique()

array(['врач-терапевт участковый', 'врач-педиатр участковый',
       'врач общей практики (семейный врач)', 'врач-офтальмолог',
       'врач-оториноларинголог', 'врач-хирург', 'врач-акушер-гинеколог',
       'врач-психиатр-нарколог', 'врач-фтизиатр участковый',
       'врач-фтизиатр', 'врач-стоматолог', 'врач-стоматолог-терапевт',
       'врач-стоматолог детский', 'врач-психиатр детский',
       'врач - детский хирург', 'врач-психиатр детский участковый'],
      dtype=object)

* Этот код определяет пользовательскую функцию `customized_nanmean` для вычисления среднего значения из заданного списка `values`, который может содержать пропущенные значения (`None` или `NaN`).

Сначала код проверяет, являются ли все значения в группе пропущенными (`None`) или NaN (`np.nan`). Для этого используется выражение `all(pd.isna(x) or x is None for x in values)`. Если все значения равны `None` или NaN, функция возвращает `None`, т.е., нет доступных данных для расчета среднего значения и возвращается специальное значение `None`.

В противном случае (если есть хотя бы одно числовое значение), функция использует `np.nanmean(values)` для вычисления среднего значения. Функция `np.nanmean` вычисляет среднее значение, игнорируя пропущенные значения (`NaN`), и возвращает результат с плавающей точкой.

И наконец, результат среднего значения округляется до ближайшего целого числа с помощью `round(..., 0)` и возвращается.

Функция вызывается при построении сводной таблицы, так что в итоге мы получаем массив средних значений за исключением `None`


In [593]:
def customized_nanmean(values):
    # Проверяем, все ли значения в группе равны None или NaN
    if all(pd.isna(x) or x is None for x in values):
        return None
    else:
        return round(np.nanmean(values), 0)

In [594]:
slide_5pivot = pd.pivot_table(analit_slide_5, index='Организация', columns='Должность', values='Минимум дней', aggfunc=customized_nanmean, margins=True, margins_name='Итог').fillna('НДЗ')

In [595]:
slide_5pivot.reset_index(drop=False, inplace=True)

In [596]:
slide_5pivot = slide_5pivot.rename_axis(index=None, columns=None)

In [597]:
slide_5pivot = slide_5pivot[slide_5pivot['Организация'] != 'ГБУЗ ТО «Перинатальный центр»'].reset_index(drop=True)

Вот тут нужно проверить результат на соответсвие с таблицей Ани флягиной чтобы понять должны ли вообще они отправлять данные или нет

In [598]:
uchet_doljnosty = pd.read_excel(ROOT_PATH + '//guide//uchet_doljnosty.xlsx').fillna(0)

In [599]:
uchet_doljnosty = uchet_doljnosty.sort_values(by='Организация', ascending=True).reset_index(drop=True)

In [600]:
len(slide_5pivot.columns)

18

In [601]:
len(uchet_doljnosty.columns)

17

In [602]:
slide_5pivot = slide_5pivot[['Организация', 'врач общей практики (семейный врач)', 'врач-акушер-гинеколог', 'врач-оториноларинголог', 
                             'врач-офтальмолог', 'врач-педиатр участковый', 'врач-психиатр-нарколог', 'врач-стоматолог', 
                             'врач-терапевт участковый', 'врач-фтизиатр участковый', 'врач-фтизиатр', 'врач-хирург', 'врач - детский хирург', 
                             'врач-стоматолог-терапевт', 'врач-стоматолог детский', 'врач-психиатр детский', 'врач-психиатр детский участковый']]

In [603]:
# Создание третьего DataFrame, в котором значения из df1 берутся, если значения в duchet_doljnosty != 0
df5 = slide_5pivot.copy()
for column in df5.columns:
    for idx, value in enumerate(df5[column]):
        df5.at[idx, column] = np.where(uchet_doljnosty.at[idx, column] != 0, slide_5pivot.at[idx, column], '-')

In [604]:
df5['Организация'] = df5['Организация'].astype(str)

In [605]:
df5 = pd.merge(df5, sequence_mo, how='right', on='Организация')

##### Расчет 8 столбца первого слайда

###### Вот тут добавляем требование заменять нулевые ставки на 1 

In [606]:
merge_ras_1c_uchet_secret['mp_stavka'] = np.where(merge_ras_1c_uchet_secret['mp_stavka'] == 0, 1, merge_ras_1c_uchet_secret['mp_stavka'])

In [607]:
merge_ras_1c_uchet_data = merge_ras_1c_uchet_secret.copy()

In [608]:
# Расчет 8 столбца первого слайда
slide_1_columns8_data = merge_ras_1c_uchet_data[['Организация', 
                                            'Должность', 
                                            'mp_id', 
                                            'mp_fio', 
                                            'mp_stavka']]
subset_data = slide_1_columns8_data.copy()
subset_data.drop_duplicates(subset=['Организация', 'Должность', 'mp_id'], inplace=True)

# Расчет 8 столбца первого слайда
slide_1_columns8 = pd.pivot_table(subset_data, 
                                  index='Организация', 
                                  values='mp_stavka', 
                                  aggfunc=np.sum, 
                                  margins=True, 
                                  margins_name='Итого').fillna(0).reset_index()
slide_1_columns8.rename(columns={'Организация': 'Наименование МО'}, inplace=True)

/tmp/ipykernel_7504/863273829.py:11: FutureWarning: The provided callable <function sum at 0x7eff90181d80> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  slide_1_columns8 = pd.pivot_table(subset_data,


##### Итоговый расчет первого слайда

In [609]:
# Итоговый расчет первого слайда
# Объединяем 
slide_1_all = [slide_1_column1, slide_1_column2, slide_1_column3, slide1_column4, slide_1_column6, slide_1_column7, slide_1_columns8]
df = reduce(lambda left, right: pd.merge(left,right,on=['Наименование МО'], how='outer'), slide_1_all)

df.rename(columns={'Всего': 'Кол-во физ-лиц в ФРМР', 'Итого': 'Кол-во физ-лиц в ФРМР за минусом ДО'}, inplace=True)
df = df.fillna(0)

# ##### Вот тут я пересчитываю количество физических лиц за минусом ДО еще раз
df['Кол-во физ-лиц в ФРМР за минусом ДО'] = df['Кол-во физ-лиц в ФРМР'] - df['Кол-во физ-лиц в ФРМР в ДО']
df['Доля физ. лиц с расписанием от общего количества физ лиц'] = round(df['Кол-во физ-лиц на которые заведено расписание'] / df['Кол-во физ-лиц в ФРМР за минусом ДО'] * 100, 1)
df.replace([np.inf , -np.inf ], 0 , inplace= True)
df = df[df['Наименование МО'] != 'Итого']
df = df.sort_values(by='Доля физ. лиц с расписанием от общего количества физ лиц', ascending=False)
df.reset_index(drop=True)
df = df.set_index('Наименование МО')
df.loc['Итого']=df.sum()
df = df.reset_index()
# Значение для поиска
index_values = 'Итого'
# Находим индекс строки с заданным значением в 'Столбец1'
index_values_value = df.loc[df['Наименование МО'] == index_values].index[0]
# Вычисляем среднее значение по столбцу 'Столбец2'
mean_procent = df['Доля физ. лиц с расписанием от общего количества физ лиц'].median()
# Изменяем значение ячейки на среднее значение
df.loc[index_values_value , 'Доля физ. лиц с расписанием от общего количества физ лиц'] = mean_procent
df['Доля физ. лиц с расписанием от общего количества физ лиц'] = round(df['Доля физ. лиц с расписанием от общего количества физ лиц'], 1)
df.rename(columns={'Кол-во должностей из 14': 'Кол-во введенных должностей в ШР МО из учитываемых 14',
                   'Кол-во физ-лиц в ФРМР за минусом ДО': 'Кол-во физ-лиц в ФРМР за минусом находящихся в отпуске по беременности и родам и по уходу за ребенком',
                   'Доля физ. лиц с расписанием от общего количества физ лиц': 'Доля физ. лиц с расписанием от общего количества физ лиц за минусом находящихся в отпуске по беременности и родам и по уходу за ребенком',
                   'Кол-во физ-лиц в ФРМР в ДО': 'Кол-во физ-лиц в ФРМР находящихся в отпуске по беременности и родам и по уходу за ребенком', 
                   'mp_stavka': 'Кол-во ставок на которое сформировано расписание'}, inplace=True)
df = df[['Наименование МО',
       'Кол-во введенных должностей в ШР МО из учитываемых 14',
       'Кол-во физ-лиц в ФРМР',
       'Кол-во физ-лиц в ФРМР за минусом находящихся в отпуске по беременности и родам и по уходу за ребенком',
       'Кол-во физ-лиц на которые заведено расписание',
       'Кол-во физ-лиц в ФРМР находящихся в отпуске по беременности и родам и по уходу за ребенком',
       'Доля физ. лиц с расписанием от общего количества физ лиц за минусом находящихся в отпуске по беременности и родам и по уходу за ребенком', 
       'Кол-во ставок в ШР МО', 
       'Кол-во ставок на которое сформировано расписание']]
df['Доля физ. лиц с расписанием от общего количества физ лиц за минусом находящихся в отпуске по беременности и родам и по уходу за ребенком'] = round(df['Кол-во физ-лиц на которые заведено расписание'] / df['Кол-во физ-лиц в ФРМР за минусом находящихся в отпуске по беременности и родам и по уходу за ребенком'] * 100, 1)

In [610]:
df['Кол-во ставок в ШР МО'] = df['Кол-во ставок в ШР МО'].round(2)

###### Создаем презентацию

In [611]:
# Создаем презентацию
prs = Presentation()
prs.slide_width = Inches(16)
prs.slide_height = Inches(9)
slide_width = prs.slide_width
slide_height = prs.slide_height

##### Создаю первый слайд

In [612]:

# Создаю первый слайд
# Указываем макет создаем слайд настраиваем параменты слайда
blank_slide_layout = prs.slide_layouts[5]
slide = prs.slides.add_slide(blank_slide_layout)
# Создаем заголовок
title_slide = slide.shapes.title
# Установка цвета фона заголовка
title_slide.fill.solid()
title_slide.fill.fore_color.rgb = color_title
title_slide.text = f'ПОКАЗАТЕЛИ ОРГАНИЗАЦИИ ЗАПИСИ В ОТЧЕТНОМ ПЕРИОДЕ ({period_last}) «ОХВАТ РАСПИСАНИЕМ СОТРУДНИКОВ»'
title_slide.height = Inches(0.8)
title_slide.width = Inches(16)
title_slide.top = Inches(0.1)
title_slide.left = Inches(0)
# Устанавливаем размер шрифта
title_slide.text_frame.paragraphs[0].runs[0].font.size = Pt(10)
# Устанавливаем цвет шрифта
title_slide.text_frame.paragraphs[0].runs[0].font.color.rgb = color_neutral 
#Создаем макет табличной формы
left = Inches(0)
width = int(slide_width * 1)
height = int (slide_height * 0.7)
top = Inches(1)
shape = slide.shapes.add_table(30, 9, left, top, width, height)
table_placeholder = slide.shapes[0]
# Создаем таблицу
table = shape.table
# Уменьшаем отступы в ячейках таблицы
for row in table.rows:
    for cell in row.cells:
        cell.margin_top = 0
        cell.margin_bottom = 0
# Заполняем заголовки столбцов
for i, column in enumerate(df.columns):
    table.cell(0, i).text = column
    cell = table.cell(0, i)
    cell.text_frame.clear()  # Очистка содержимого ячейки
    cell.text_frame.auto_size = MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE  # Масштабирование текста
    p = cell.text_frame.paragraphs[0]
    p.text = column
    p.alignment = PP_ALIGN.CENTER
    p.font.size = Pt(10)
    p.font.name = 'Calibri'
    cell.fill.solid()
    cell.fill.fore_color.rgb = color_header  # Задание цвета фона ячейки

for i, row in enumerate(df.itertuples(), start=1):
    for j, value in enumerate(row[1:], start=0):
        table.cell(i, j).text = str(value)

for i, row in enumerate(df.itertuples(), start=1):
    for j, value in enumerate(row[1:], start=0):
        cell = table.cell(i, j)
        cell.text_frame.clear()  # Очистка содержимого ячейки
        cell.text_frame.auto_size = MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE  # Масштабирование текста
        cell.width = Inches(0.3)  # Ширина ячейки
        cell.height = Inches(0.6)  # Высота ячейки
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral  # Задание цвета фона ячейки
        p = cell.text_frame.paragraphs[0]
        p.text = str(value)
        p.alignment = PP_ALIGN.CENTER
        p.font.size = Pt(10)
        p.font.name = 'Calibri'

# Слайд 1 Столбец 7
column_index = 6
column_name = df.columns[column_index]
# Получение соответствующего столбца из DataFrame
column = df[column_name]
# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table.cell(i, column_index)
    # Раскраска ячейки на основе значения из DataFrame
    if value >= 100:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_green  
    elif 80 <= value < 100:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_yallow  
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red

##### Создание слайда 2

In [613]:
# Создание слайда 2
# Указываем макет создаем слайд настраиваем параменты слайда
blank_slide_layout = prs.slide_layouts[5]
slide2 = prs.slides.add_slide(blank_slide_layout)
slide_width = prs.slide_width
slide_height = prs.slide_height

slide2.slide_width = slide_width
slide2.slide_height = slide_height
# Создаем заголовок
title_slide2 = slide2.shapes.title
title_slide2.text = f'ПОКАЗАТЕЛИ ОРГАНИЗАЦИИ ЗАПИСИ В ОТЧЕТНОМ ПЕРИОДЕ ({period}) «ПОЛНОТА ВЕДЕНИЯ РАСПИСАНИЯ И КОНКУРЕНТНОСТЬ»'
title_slide2.height = Inches(0.9)
title_slide2.width = Inches(16)
title_slide2.top = Inches(0.1)
title_slide2.left = Inches(0)
# Установка цвета фона заголовка
title_slide2.fill.solid()
title_slide2.fill.fore_color.rgb = color_title
# Устанавливаем размер шрифта
title_slide2.text_frame.paragraphs[0].runs[0].font.size = Pt(14)
# Устанавливаем цвет шрифта
title_slide2.text_frame.paragraphs[0].runs[0].font.color.rgb = color_neutral
# Создаем макет табличной формы
left = Inches(0.1)
width = int(slide_width * 0.98)
height = int (slide_height * 0.8)
top = Inches(1.1)
shape2 = slide2.shapes.add_table(30, 6, left, top, width, height)
table_placeholder2 = slide2.shapes[0]
# Создаем таблицу
table2 = shape2.table
# Уменьшаем отступы в ячейках таблицы
for row in table2.rows:
    for cell in row.cells:
        cell.margin_top = 0
        cell.margin_bottom = 0
# Заполняем заголовки столбцов
for i, column in enumerate(df2.columns):
    table2.cell(0, i).text = column
    cell = table2.cell(0, i)
    cell.text_frame.clear()  # Очистка содержимого ячейки
    cell.text_frame.auto_size = MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE  # Масштабирование текста
    p = cell.text_frame.paragraphs[0]
    p.text = column
    p.alignment = PP_ALIGN.CENTER
    p.font.size = Pt(12)
    p.font.name = 'Calibri'
    cell.fill.solid()
    cell.fill.fore_color.rgb = color_header  # Задание цвета фона ячейки

for i, row in enumerate(df2.itertuples(), start=1):
    for j, value in enumerate(row[1:], start=0):
        table2.cell(i, j).text = str(value)

for i, row in enumerate(df2.itertuples(), start=1):
    for j, value in enumerate(row[1:], start=0):
        cell = table2.cell(i, j)
        cell.text_frame.clear()  # Очистка содержимого ячейки
        cell.text_frame.auto_size = MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE  # Масштабирование текста
        cell.width = Inches(0.3)  # Ширина ячейки
        cell.height = Inches(0.6)  # Высота ячейки
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral  # Задание цвета фона ячейки
        p = cell.text_frame.paragraphs[0]
        p.text = str(value)
        p.alignment = PP_ALIGN.CENTER
        p.font.size = Pt(10)
        p.font.name = 'Calibri'
# Столбец 6
column_index = 5
column_name = df2.columns[column_index]
# Получение соответствующего столбца из DataFrame
column = df2[column_name]
# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table2.cell(i, column_index)
    # Раскраска ячейки на основе значения из DataFrame
    if value >= 65:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_green  
    elif 60 <= value < 65:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_yallow
    elif value == 0:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red  


# Столбец 5
column_index = 4
column_name = df2.columns[column_index]
# Получение соответствующего столбца из DataFrame
column = df2[column_name]
# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table2.cell(i, column_index)
    # Раскраска ячейки на основе значения из DataFrame
    if value >= 65:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_green  
    elif 60 <= value < 65:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_yallow  
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red  

# Столбец 4
column_index = 3
column_name = df2.columns[column_index]
# Получение соответствующего столбца из DataFrame
column = df2[column_name]
# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table2.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value >= 80:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_green  # Зеленый цвет
    elif 60 <= value < 80:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_yallow  # Желтый цвет
    elif value == 0:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red  # Красный цвет

# Столбец 3
column_index = 2
column_name = df2.columns[column_index]

# Получение соответствующего столбца из DataFrame
column = df2[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table2.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value >= 65:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_green  # Зеленый цвет
    elif 60 <= value < 65:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_yallow  # Зеленый цвет
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red  # Красный цвет
# Столбец 2
column_index = 1
column_name = df2.columns[column_index]

# Получение соответствующего столбца из DataFrame
column = df2[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table2.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value >= 80:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_green  # Зеленый цвет
    elif 60 <= value < 80:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_yallow 
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red  # Красный цвет

##### Создаем слайд 3

In [614]:
# Создаем слайд 3
# Указываем макет создаем слайд настраиваем параменты слайда
blank_slide_layout = prs.slide_layouts[5]
slide3 = prs.slides.add_slide(blank_slide_layout)
slide_width = prs.slide_width
slide_height = prs.slide_height
slide3.slide_width = slide_width
slide3.slide_height = slide_height

# Создаем заголовок
title_slide3 = slide3.shapes.title
title_slide3.text = f'ПОКАЗАТЕЛИ ОРГАНИЗАЦИИ ЗАПИСИ В ОТЧЕТНОМ ПЕРИОДЕ ({period}) «ЭФФЕКТИВНОСТЬ ФОРМИРОВАНИЯ РАСПИСАНИЯ»'
title_slide3.height = Inches(0.9)
title_slide3.width = Inches(16)
title_slide3.top = Inches(0.1)
title_slide3.left = Inches(0)

# Установка цвета фона заголовка
title_slide3.fill.solid()
title_slide3.fill.fore_color.rgb = color_title
# Устанавливаем размер шрифта
title_slide3.text_frame.paragraphs[0].runs[0].font.size = Pt(14)
# Устанавливаем цвет шрифта
title_slide3.text_frame.paragraphs[0].runs[0].font.color.rgb = color_neutral
# Создаем макет табличной формы
left = Inches(0.1)
width = int(slide_width * 0.98)
height = int (slide_height * 0.85)
top = Inches(1.1)
shape3 = slide3.shapes.add_table(30, 5, left, top, width, height)
table_placeholder3 = slide3.shapes[0]
# Создаем таблицу
table3 = shape3.table
# Уменьшаем отступы в ячейках таблицы
for row in table3.rows:
    for cell in row.cells:
        cell.margin_top = 0
        cell.margin_bottom = 0
# Заполняем заголовки столбцов
for i, column in enumerate(df3.columns):
    table3.cell(0, i).text = column
    cell = table3.cell(0, i)
    cell.text_frame.clear()  # Очистка содержимого ячейки
    cell.text_frame.auto_size = MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE  # Масштабирование текста
    p = cell.text_frame.paragraphs[0]
    p.text = column
    p.alignment = PP_ALIGN.CENTER
    p.font.size = Pt(10)
    p.font.name = 'Calibri'
    cell.fill.solid()
    cell.fill.fore_color.rgb = color_header  # Задание цвета фона заголовка

for i, row in enumerate(df3.itertuples(), start=1):
    for j, value in enumerate(row[1:], start=0):
        table3.cell(i, j).text = str(value)

for i, row in enumerate(df3.itertuples(), start=1):
    for j, value in enumerate(row[1:], start=0):
        cell = table3.cell(i, j)
        cell.text_frame.clear()  # Очистка содержимого ячейки
        cell.text_frame.auto_size = MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE  # Масштабирование текста
        cell.width = Inches(0.3)  # Ширина ячейки
        cell.height = Inches(0.6)  # Высота ячейки
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral  # Задание цвета фона ячейки
        p = cell.text_frame.paragraphs[0]
        p.text = str(value)
        p.alignment = PP_ALIGN.CENTER
        p.font.size = Pt(10)
        p.font.name = 'Calibri'


# Столбец 2
column_name = 'Занятость слотов всего (доля от общего числа созданных и количества занятых)'
column_index = 1

# Получение соответствующего столбца из DataFrame
column = df3[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table3.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value >= 80:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_green  # Зеленый цвет
    elif 50 <= value < 80:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_yallow  # Нейтральный цвет
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red  # Красный цвет
# Столбец 3
column_name = 'Доля неявок от общего числа записей'
column_index = 2

# Получение соответствующего столбца из DataFrame
column = df3[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table3.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value < 20:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_green  # Зеленый цвет
    elif 20 <= value <= 30:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_yallow  # Нейтральный цвет
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red  # Красный цвет


# Столбец 4
column_name = 'Доля пациентов, принятых без предварительной записи'
column_index = 3

# Получение соответствующего столбца из DataFrame
column = df3[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table3.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value < 20:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_green  # Зеленый цвет
    elif 20 <= value <= 30:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_yallow  # Нейтральный цвет
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red  # Красный цвет


# Столбец 5
column_name = 'Доля посещений на дому'
column_index = 4

# Получение соответствующего столбца из DataFrame
column = df3[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table3.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value == 0:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_yallow  # Нейтральный цвет
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red  # Красный цвет

In [615]:
df4 = df4[df4['Наименование МО'] != 0]

##### Создаем 4 слайд

In [616]:
# Создаем 4 слайд
# Указываем макет создаем слайд настраиваем параменты слайда
blank_slide_layout = prs.slide_layouts[5]
slide4 = prs.slides.add_slide(blank_slide_layout)

slide_width = prs.slide_width
slide_height = prs.slide_height

slide4.slide_width = slide_width
slide4.slide_height = slide_height

# Создаем заголовок
title_slide4 = slide4.shapes.title
title_slide4.text = f'ЭФФЕКТИВНОСТЬ ОРГАНИЗАЦИИ ОБРАБОТКИ И РЕГИСТРАЦИИ ИНФОРМАЦИИ ОБ ОБРАЩЕНИЯХ ЗАПИСИ ОБРАТНАЯ СВЯЗЬ ПО ЗАПИСИ В ОТЧЕТНОМ ПЕРИОДЕ ({period})'
title_slide4.height = Inches(0.9)
title_slide4.width = Inches(16)
title_slide4.top = Inches(0.1)
title_slide4.left = Inches(0)

# Установка цвета фона заголовка
title_slide4.fill.solid()
title_slide4.fill.fore_color.rgb = color_title
# Устанавливаем размер шрифта
title_slide4.text_frame.paragraphs[0].runs[0].font.size = Pt(14)
# Устанавливаем цвет шрифта
title_slide4.text_frame.paragraphs[0].runs[0].font.color.rgb = color_neutral
# Создаем макет табличной формы
left = Inches(0)
width = int(slide_width)
height = int (slide_height * 0.8)
top = Inches(1.1)
shape4 = slide4.shapes.add_table(30, 6, left, top, width, height)
table_placeholder4 = slide4.shapes[0]
# Создаем таблицу
table4 = shape4.table
# Уменьшаем отступы в ячейках таблицы
for row in table4.rows:
    for cell in row.cells:
        cell.margin_top = 0
        cell.margin_bottom = 0
# Заполняем заголовки столбцов
for i, column in enumerate(df4.columns):
    table4.cell(0, i).text = column
    cell = table4.cell(0, i)
    cell.text_frame.clear()  # Очистка содержимого ячейки
    cell.text_frame.auto_size = MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE  # Масштабирование текста
    p = cell.text_frame.paragraphs[0]
    p.text = column
    p.alignment = PP_ALIGN.CENTER
    p.font.size = Pt(12)
    p.font.name = 'Calibri'
    cell.fill.solid()
    cell.fill.fore_color.rgb = color_header  # Задание цвета фона ячейки заголовка

for i, row in enumerate(df4.itertuples(), start=1):
    for j, value in enumerate(row[1:], start=0):
        table4.cell(i, j).text = str(value)

for i, row in enumerate(df4.itertuples(), start=1):
    for j, value in enumerate(row[1:], start=0):
        cell = table4.cell(i, j)
        cell.text_frame.clear()  # Очистка содержимого ячейки
        cell.text_frame.auto_size = MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE  # Масштабирование текста
        cell.width = Inches(0.3)  # Ширина ячейки
        cell.height = Inches(0.6)  # Высота ячейки
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral  # Задание цвета фона ячейки
        p = cell.text_frame.paragraphs[0]
        p.text = str(value)
        p.alignment = PP_ALIGN.CENTER
        p.font.size = Pt(10)
        p.font.name = 'Calibri'




In [617]:
# Имя столбца для итерации
column_name = 'Движение лиц в листе ожидания'

column_index = 5

# Получение соответствующего столбца из DataFrame
column = df4[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table4.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value <= 0:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_green  # Зеленый цвет
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red  # Красный цвет

In [618]:
# Имя столбца для итерации
column_name = 'Доля лиц, включенных в лист ожидания от общего количества конкурентной записи'

column_index = 4

# Получение соответствующего столбца из DataFrame
column = df4[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table4.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value < 1:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_green  # Зеленый цвет
    elif value >= 1 <= 5:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_yallow # Зеленый цвет
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red  # Красный цвет

##### Создаем пятый слайд

In [619]:
# Создаем пятый слайд
blank_slide_layout = prs.slide_layouts[5]
slide5 = prs.slides.add_slide(blank_slide_layout)
slide_width = prs.slide_width
slide_height = prs.slide_height
slide5.slide_width = slide_width
slide5.slide_height = slide_height
# Создаем заголовок
title_slide5 = slide5.shapes.title
title_slide5.text = f'СРЕДНЯЯ ДЛИТЕЛЬНОСТЬ ОЖИДАНИЯ (кол-во дней) В ОТЧЕТНОМ ПЕРИОДЕ ({period})'
title_slide5.height = Inches(0.6)
title_slide5.width = Inches(16)
title_slide5.top = Inches(0.1)
title_slide5.left = Inches(0)
# Установка цвета фона заголовка
title_slide5.fill.solid()
title_slide5.fill.fore_color.rgb = color_title
# Устанавливаем размер шрифта
title_slide5.text_frame.paragraphs[0].runs[0].font.size = Pt(14)
# Устанавливаем цвет шрифта
title_slide5.text_frame.paragraphs[0].runs[0].font.color.rgb = color_neutral 
# Создаем макет табличной формы
left = Inches(0.5)
width = int(slide_width * 0.8)
height = int(slide_height * 0.8)
top = Inches(0.7)
shape5 = slide5.shapes.add_table(30, 17, left, top, width, height)
table_placeholder5 = slide5.shapes[0]
# Создаем таблицу
table5 = shape5.table
# Уменьшаем отступы в ячейках таблицы
for row in table5.rows:
    for cell in row.cells:
        cell.margin_top = 0
        cell.margin_bottom = 0
# Заполняем заголовки столбцов
for i, column in enumerate(df5.columns):
    table5.cell(0, i).text = column
    cell = table5.cell(0, i)
    cell.text_frame.clear()  # Очистка содержимого ячейки
    cell.text_frame.auto_size = MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE  # Масштабирование текста
    p = cell.text_frame.paragraphs[0]
    p.text = column
    p.alignment = PP_ALIGN.CENTER
    p.font.size = Pt(10)
    p.font.name = 'Calibri'
    cell.fill.solid()
    cell.fill.fore_color.rgb = color_header  # Задание цвета фона ячейки

for i, row in enumerate(df5.itertuples(), start=1):
    for j, value in enumerate(row[1:], start=0):
        table5.cell(i, j).text = str(value)

for i, row in enumerate(df5.itertuples(), start=1):
    for j, value in enumerate(row[1:], start=0):
        cell = table5.cell(i, j)
        cell.text_frame.clear()  # Очистка содержимого ячейки
        cell.text_frame.auto_size = MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE  # Масштабирование текста
        cell.width = int(slide_width * 0.05)  # Ширина ячейки
        cell.height = int(slide_height * 0.02)  # Высота ячейки
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral  # Задание цвета фона ячейки
        p = cell.text_frame.paragraphs[0]
        p.text = str(value)
        p.alignment = PP_ALIGN.CENTER
        p.font.size = Pt(9)
        p.font.name = 'Calibri'

# Итерация по ячейкам таблицы и раскраска по условию
for row in table5.rows:
    for cell in row.cells:
        # Условие для раскраски ячеек в красный цвет, если значение ячейки равно "НДЗ"
        if cell.text == "НДЗ":
            fill = cell.fill
            fill.solid()
            fill.fore_color.rgb = color_red  # Красный цвет

# Получаем объект столбца по его номеру
table5.columns[0].width = Inches(2)

exception_list_1 = ['0.0', '1.0', '-']
exception_list_2 = ['0.0', '1.0', '2.0', '3.0', '4.0', '5.0', '6.0', '7.0', '8.0', '9.0', '10.0', '11.0', '12.0', '13.0', '14.0','-']
# ###### Раскраска по столбцам
# Столбец 1
column_name = 'врач общей практики (семейный врач)'
column_index = 1

# Получение соответствующего столбца из DataFrame
column = df5[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table5.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value not in exception_list_1:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral


In [620]:

# Столбец 2
column_name = 'врач-акушер-гинеколог'

column_index = 2

# Получение соответствующего столбца из DataFrame
column = df5[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table5.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value not in exception_list_2:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral

# Столбец 3
column_name = 'врач-оториноларинголог'

column_index = 3

# Получение соответствующего столбца из DataFrame
column = df5[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table5.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value not in exception_list_2:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral

# Столбец 4
column_name = 'врач-офтальмолог'
column_index = 4

# Получение соответствующего столбца из DataFrame
column = df5[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table5.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value not in exception_list_2:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral

# Столбец 5
column_name = 'врач-педиатр участковый'
column_index = 5

# Получение соответствующего столбца из DataFrame
column = df5[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table5.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value not in exception_list_1:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral

# Столбец 6
column_name = 'врач-психиатр-нарколог'
column_index = 6

# Получение соответствующего столбца из DataFrame
column = df5[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table5.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value not in exception_list_2:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral

# Столбец 7
column_name = 'врач-стоматолог'
column_index = 7

# Получение соответствующего столбца из DataFrame
column = df5[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table5.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value not in exception_list_2:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral

# Столбец 8
column_name = 'врач-терапевт участковый'
column_index = 8

# Получение соответствующего столбца из DataFrame
column = df5[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table5.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value not in exception_list_1:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral

# Столбец 9
column_name = 'врач-фтизиатр участковый'
column_index = 9

# Получение соответствующего столбца из DataFrame
column = df5[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table5.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value not in exception_list_2:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral

# Столбец 10
column_name = 'врач-фтизиатр'
column_index = 10

# Получение соответствующего столбца из DataFrame
column = df5[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table5.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value not in exception_list_2:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral

# Столбец 11
column_name = 'врач-хирург'
column_index = 11

# Получение соответствующего столбца из DataFrame
column = df5[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table5.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value not in exception_list_2:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral

# Столбец 12
column_name = 'врач - детский хирург'
column_index = 12

# Получение соответствующего столбца из DataFrame
column = df5[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table5.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value not in exception_list_2:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral

# Столбец 13
column_name = 'врач-стоматолог-терапевт'
column_index = 13

# Получение соответствующего столбца из DataFrame
column = df5[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table5.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value not in exception_list_2:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral

# Столбец 14
column_name = 'врач-стоматолог детский'
column_index = 14

# Получение соответствующего столбца из DataFrame
column = df5[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table5.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value not in exception_list_2:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral


# Столбец 15
column_name = 'врач-психиатр детский'
column_index = 15

# Получение соответствующего столбца из DataFrame
column = df5[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table5.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value not in exception_list_2:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral

# Столбец 16
column_name = 'врач-психиатр детский участковый'
column_index = 16

# Получение соответствующего столбца из DataFrame
column = df5[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table5.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value not in exception_list_2:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral

##### Формируем колонки 6 слайда

In [621]:
# Формируем колонки 6 слайда
sequence_mo3 = sequence_mo2.copy()
sequence_mo3.loc[len(sequence_mo3)] = 'Итого'
slide_6_column1 = df[['Наименование МО', 'Доля физ. лиц с расписанием от общего количества физ лиц за минусом находящихся в отпуске по беременности и родам и по уходу за ребенком']]
slide_6_column1 = pd.merge(sequence_mo3, slide_6_column1, how='left', on='Наименование МО')
slide_6_column2 = itog_slots
slide_6_column2.rename(columns={'Организация': 'Наименование МО'}, inplace=True)
slide_6_column2 = pd.merge(sequence_mo3, slide_6_column2, how='left', on='Наименование МО')
slide_6_column3 = df2[['Наименование МО', 'Доля конкурентных слотов от общего количества слотов, не менее 65%']]
slide_6_column4 = result_slide3[['Наименование МО', 'Занятость слотов всего (доля от общего числа созданных и количества занятых)']]
slide_6_column4 = pd.merge(sequence_mo3, slide_6_column4, how='left', on='Наименование МО')
slide_6_column5 = result_slide3[['Наименование МО', 
                                 'Доля пациентов, принятых без предварительной записи']]
slide_6_column5 = pd.merge(sequence_mo3, slide_6_column5, how='left', on='Наименование МО')
slide_6_column6 = result_slide4[['Наименование МО', 'Движение лиц в листе ожидания']]
slide_6_column6 = pd.merge(sequence_mo3, slide_6_column6, how='left', on='Наименование МО')
# Агрегируем данные по колонкам
slide_6_all = [slide_6_column1, 
               slide_6_column2, 
               slide_6_column3, 
               slide_6_column4, 
               slide_6_column5, 
               slide_6_column6]
df6 = reduce(lambda left, right: pd.merge(left,right,on=['Наименование МО'], how='outer'), slide_6_all)

##### Создаем шестой слайд

In [622]:
#Создаем шестой слайд

# Указываем макет создаем слайд настраиваем параменты слайда

blank_slide_layout = prs.slide_layouts[5]
slide6 = prs.slides.add_slide(blank_slide_layout)
slide_width = prs.slide_width
slide_height = prs.slide_height
slide6.slide_width = slide_width
slide6.slide_height = slide_height
# Создаем заголовок

title_slide6 = slide6.shapes.title

title_slide6.text = 'СИСТЕМА КОНТРОЛЯ - РАБОТА С ВЫРАВНИВАНИЕМ ПОКАЗАТЕЛЕЙ'
title_slide6.height = Inches(0.6)
title_slide6.width = Inches(16)
title_slide6.top = Inches(0.1)
title_slide6.left = Inches(0)
# Установка цвета фона заголовка
title_slide6.fill.solid()
title_slide6.fill.fore_color.rgb = color_title
# Устанавливаем размер шрифта
title_slide6.text_frame.paragraphs[0].runs[0].font.size = Pt(12)
# Устанавливаем цвет шрифта
title_slide6.text_frame.paragraphs[0].runs[0].font.color.rgb = color_neutral 
# Создаем макет табличной формы
left = Inches(0.5)
width = int(slide_width * 0.8)
height = int(slide_height * 0.8)
top = Inches(0.7)
shape6 = slide6.shapes.add_table(30, 7, left, top, width, height)
table_placeholder6 = slide6.shapes[0]
# Создаем таблицу
table6 = shape6.table
# Уменьшаем отступы в ячейках таблицы
for row in table6.rows:
    for cell in row.cells:
        cell.margin_top = 0
        cell.margin_bottom = 0
# Заполняем заголовки столбцов
for i, column in enumerate(df6.columns):
    table6.cell(0, i).text = column
    cell = table6.cell(0, i)
    cell.text_frame.clear()  # Очистка содержимого ячейки
    cell.text_frame.auto_size = MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE  # Масштабирование текста
    p = cell.text_frame.paragraphs[0]
    p.text = column
    p.alignment = PP_ALIGN.CENTER
    p.font.size = Pt(10)
    p.font.name = 'Calibri'
    cell.fill.solid()
    cell.fill.fore_color.rgb = color_header  # Задание цвета фона ячейки

for i, row in enumerate(df6.itertuples(), start=1):
    for j, value in enumerate(row[1:], start=0):
        table6.cell(i, j).text = str(value)

for i, row in enumerate(df6.itertuples(), start=1):
    for j, value in enumerate(row[1:], start=0):
        cell = table6.cell(i, j)
        cell.text_frame.clear()  # Очистка содержимого ячейки
        cell.text_frame.auto_size = MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE  # Масштабирование текста
        cell.width = int(slide_width * 0.05)  # Ширина ячейки
        cell.height = int(slide_height * 0.02)  # Высота ячейки
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral  # Задание цвета фона ячейки
        p = cell.text_frame.paragraphs[0]
        p.text = str(value)
        p.alignment = PP_ALIGN.CENTER
        p.font.size = Pt(9)
        p.font.name = 'Calibri'

# Слайд 1 Столбец 1
column_index = 1
column_name = df6.columns[column_index]
# Получение соответствующего столбца из DataFrame
column = df6[column_name]
# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table6.cell(i, column_index)
    # Раскраска ячейки на основе значения из DataFrame
    if value >= 100:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_green  
    elif 80 <= value < 100:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_yallow  
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red

# Слайд 1 Столбец 2
column_index = 2
column_name = df6.columns[column_index]
# Получение соответствующего столбца из DataFrame
column = df6[column_name]
# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table6.cell(i, column_index)
    # Раскраска ячейки на основе значения из DataFrame
    if value >= 80:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_green  
    elif 60 <= value < 80:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_yallow  
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red

# Слайд 1 Столбец 3
column_index = 3
column_name = df6.columns[column_index]
# Получение соответствующего столбца из DataFrame
column = df6[column_name]
# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table6.cell(i, column_index)
    # Раскраска ячейки на основе значения из DataFrame
    if value >= 65:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_green  
    elif 60 <= value < 65:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_yallow  
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red

# Слайд 1 Столбец 4
column_index = 4
column_name = df6.columns[column_index]
# Получение соответствующего столбца из DataFrame
column = df6[column_name]
# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table6.cell(i, column_index)
    # Раскраска ячейки на основе значения из DataFrame
    if value >= 80:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_green  
    elif 50 <= value < 80:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_yallow  
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red

# Слайд 1 Столбец 5
column_index = 5
column_name = df6.columns[column_index]
# Получение соответствующего столбца из DataFrame
column = df6[column_name]
# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table6.cell(i, column_index)
    # Раскраска ячейки на основе значения из DataFrame
    if value > 30:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red
    elif 20 < value <= 30:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_yallow  
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_green

# Слайд 1 Столбец 6
column_index = 6
column_name = df6.columns[column_index]
# Получение соответствующего столбца из DataFrame
column = df6[column_name]
# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table6.cell(i, column_index)
    # Раскраска ячейки на основе значения из DataFrame
    if value <= 0:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_green    
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red



In [623]:
prs.save('/home/serg/prezentation/output/ПрезентацияИ38.pptx')

##### Таблица по штатным единицам

* Создаем объединенный файл на основе выгрузки ФРМР

In [624]:
merge_frmr_to_month = frmr_all_uchet.copy()
merge_frmr_to_month = merge_frmr_to_month[['Организация', 'Структурное подразделение', 'Должность', 'Штатных единиц', 'Занятых штатных единиц', 'В т.ч. внешнее совместительство', 'Вакансий']]
merge_frmr_to_month.to_excel('/home/serg/prezentation/output/Объединенный.xlsx', index=False)

* Считаем штатные единицы и вакансии

In [625]:
slide7_input = frmr_all_uchet.copy()
slide7_input = slide7_input[['Организация', 'Должность', 'Штатных единиц', 'Вакансий']]
# Группируем данные по столбцам 'Организация' и 'Должность' и суммируем значения в столбце 'Занятых штатных единиц'
slide7_post = slide7_input.groupby(['Организация', 'Должность'])['Штатных единиц'].sum().reset_index()
# Группируем данные по столбцам 'Организация' и 'Должность' и суммируем значения в столбце 'Вакансий'
slide7_vacancy = slide7_input.groupby(['Организация', 'Должность'])['Вакансий'].sum().reset_index()
list_post = ['Штатных единиц', 'Вакансий']
slide7_vp = slide7_input.groupby(['Организация', 'Должность'])[list_post].sum().reset_index()

* Объединяем это с данными расписания

In [626]:
slide7_tree = pd.merge(slide7_vp, employee_with_schedule, how='left', on=['Организация', 'Должность']).fillna(0)
slide7_tree['Дата'] = end_date
slide7_tree['Дата'] = pd.to_datetime(slide7_tree['Дата'], format='%Y-%m-%d')
slide7_tree['Дата'] = slide7_tree['Дата'] + pd.Timedelta(days=1)
slide7_tree.rename(columns={'кол-во сотрудников с сформированным расписанием на 7 дней': 'Сотрудников с расписанием на 7 дней'}, inplace=True)

In [627]:
slide7_tree['День'] = slide7_tree['Дата'].dt.day

In [628]:
slide7_tree['Месяц'] = slide7_tree['Дата'].dt.month

In [629]:
slide7_tree['Год'] = slide7_tree['Дата'].dt.year

In [630]:
slide7_tree.to_excel('/home/serg/prezentation/output/resultI38.xlsx', index=False)